# REST URL ONLY: Scrape Twitter for US Governors in 2022 Mid-Terms

# Setup

In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import requests
import json
from pprint import pprint

In [51]:
import tweepy
import gc #collecting garbage
from wordcloud import WordCloud,STOPWORDS, ImageColorGenerator #visualisation

sns.set_theme(context='notebook', style='whitegrid', palette='vlag', font='serif')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Get Data

In [52]:
govs_df = pd.read_csv('twitter_2022midterm_us_governors.csv', index_col=[0])
govs_df.head()
govs_df.info()

,Republican,Democrat
State,,
Wisconsin:,https://twitter.com/michelsforgov,https://twitter.com/GovEvers
Orgeon:,https://twitter.com/ChristineDrazan,https://twitter.com/TinaKotek
Nevada:,https://twitter.com/JoeLombardoNV,https://twitter.com/GovSisolak
Arizona:,https://twitter.com/KariLake,https://twitter.com/katiehobbs
Kansas:,https://twitter.com/DerekSchmidtKS,https://twitter.com/LauraKellyKS


<class 'pandas.core.frame.DataFrame'>
Index: 35 entries, Wisconsin: to Idaho:
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Republican  35 non-null     object
 1   Democrat    35 non-null     object
dtypes: object(2)
memory usage: 840.0+ bytes


In [53]:
# Clean imported DataFrame

# Lowercase Columns
govs_df.rename(columns={'Republican':'republican', 'Democrat':'democrat'}, inplace=True)


# Move State Index to separate colum for joining
state_orig_ls = govs_df.index.to_list()
# state_orig_ls[:5]

state_clean_ls = [x.lower().strip(':') for x in state_orig_ls]
# state_clean_ls[:5]

# Insert state col in first position
govs_df.insert(loc=0, column='state', value=state_clean_ls)
# govs_df['state'] = state_clean_ls

# Drop the State Index
govs_df.reset_index(drop=True, inplace=True)

govs_df.head()


,state,republican,democrat
0,wisconsin,https://twitter.com/michelsforgov,https://twitter.com/GovEvers
1,orgeon,https://twitter.com/ChristineDrazan,https://twitter.com/TinaKotek
2,nevada,https://twitter.com/JoeLombardoNV,https://twitter.com/GovSisolak
3,arizona,https://twitter.com/KariLake,https://twitter.com/katiehobbs
4,kansas,https://twitter.com/DerekSchmidtKS,https://twitter.com/LauraKellyKS


In [54]:
# Split into Two (1) Dem and (2) Rep tables

govs_d_df = govs_df[['state','democrat']].copy(deep=True)
# Extract username from url
govs_d_df['screen_name'] = govs_df["democrat"].apply(lambda x: str(x).split('/')[-1])
govs_d_df.head()

govs_r_df = govs_df[['state','republican']].copy(deep=True) 
# Extract username from url
govs_r_df['screen_name'] = govs_df["republican"].apply(lambda x: str(x).split('/')[-1])
govs_r_df.head()

,state,democrat,screen_name
0,wisconsin,https://twitter.com/GovEvers,GovEvers
1,orgeon,https://twitter.com/TinaKotek,TinaKotek
2,nevada,https://twitter.com/GovSisolak,GovSisolak
3,arizona,https://twitter.com/katiehobbs,katiehobbs
4,kansas,https://twitter.com/LauraKellyKS,LauraKellyKS


,state,republican,screen_name
0,wisconsin,https://twitter.com/michelsforgov,michelsforgov
1,orgeon,https://twitter.com/ChristineDrazan,ChristineDrazan
2,nevada,https://twitter.com/JoeLombardoNV,JoeLombardoNV
3,arizona,https://twitter.com/KariLake,KariLake
4,kansas,https://twitter.com/DerekSchmidtKS,DerekSchmidtKS


# [SKIP] Option 1: Tweepy: v1.0 API

In [7]:
import config_twitter

api_key = config_twitter.API_KEY #Your API/Consumer key 
api_secret = config_twitter.API_KEY_SECRET #Your API/Consumer Secret Key
bearer_token = config_twitter.BEARER_TOKEN # Your API/Bearer Token
access_token = config_twitter.ACCESS_TOKEN    #Your Access token key
access_token_secret = config_twitter.ACCESS_TOKEN_SECRET #Your Access token Secret key

## OAuth 1.0

In [8]:
# OAuth 1.0a

auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)
count = 0
error = 0

In [9]:
# Set default Tweepy return data type
# https://dev.to/pavangole/how-to-get-data-in-json-format-from-tweepyclient--5897
 
tweepy.Client(return_type=dict)

In [10]:
# Test 

tweepy_user_obj = api.get_user(screen_name='GovEvers')
# json.dumps(tweepy_user_obj)
type(tweepy_user_obj)
tweepy_user_obj

tweepy.models.User

User(_api=<tweepy.api.API object at 0x0000018D2D8C96A0>, _json={'id': 234896532, 'id_str': '234896532', 'name': 'Governor Tony Evers', 'screen_name': 'GovEvers', 'location': 'Madison, Wisconsin', 'profile_location': None, 'description': 'Husband to Kathy, lifelong educator, Euchre and bobblehead enthusiast, and 46th Governor of the State of Wisconsin.', 'url': 'https://t.co/JyH2JWfeyt', 'entities': {'url': {'urls': [{'url': 'https://t.co/JyH2JWfeyt', 'expanded_url': 'https://evers.wi.gov/', 'display_url': 'evers.wi.gov', 'indices': [0, 23]}]}, 'description': {'urls': []}}, 'protected': False, 'followers_count': 105573, 'friends_count': 245, 'listed_count': 902, 'created_at': 'Thu Jan 06 21:05:10 +0000 2011', 'favourites_count': 167, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': True, 'statuses_count': 7951, 'lang': None, 'status': {'created_at': 'Wed Nov 09 18:37:56 +0000 2022', 'id': 1590413394724913152, 'id_str': '1590413394724913152', 'text': 'Still jazzed

In [11]:
dir(tweepy_user_obj)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_api',
 '_json',
 'contributors_enabled',
 'created_at',
 'default_profile',
 'default_profile_image',
 'description',
 'entities',
 'favourites_count',
 'follow',
 'follow_request_sent',
 'follower_ids',
 'followers',
 'followers_count',
 'following',
 'friends',
 'friends_count',
 'geo_enabled',
 'has_extended_profile',
 'id',
 'id_str',
 'is_translation_enabled',
 'is_translator',
 'lang',
 'list_memberships',
 'list_ownerships',
 'list_subscriptions',
 'listed_count',
 'lists',
 'location',
 'name',
 'notifications',
 'parse',
 'parse_list',
 'profile_background_color',
 'profile_backgr

In [12]:
# Pretty Print JSON result from query
 
pprint(tweepy_user_obj._json)

{'contributors_enabled': False,
 'created_at': 'Thu Jan 06 21:05:10 +0000 2011',
 'default_profile': False,
 'default_profile_image': False,
 'description': 'Husband to Kathy, lifelong educator, Euchre and bobblehead '
                'enthusiast, and 46th Governor of the State of Wisconsin.',
 'entities': {'description': {'urls': []},
              'url': {'urls': [{'display_url': 'evers.wi.gov',
                                'expanded_url': 'https://evers.wi.gov/',
                                'indices': [0, 23],
                                'url': 'https://t.co/JyH2JWfeyt'}]}},
 'favourites_count': 167,
 'follow_request_sent': False,
 'followers_count': 105573,
 'following': False,
 'friends_count': 245,
 'geo_enabled': False,
 'has_extended_profile': False,
 'id': 234896532,
 'id_str': '234896532',
 'is_translation_enabled': False,
 'is_translator': False,
 'lang': None,
 'listed_count': 902,
 'location': 'Madison, Wisconsin',
 'name': 'Governor Tony Evers',
 'notifications

In [13]:
tweepy_user_obj._json.keys()

dict_keys(['id', 'id_str', 'name', 'screen_name', 'location', 'profile_location', 'description', 'url', 'entities', 'protected', 'followers_count', 'friends_count', 'listed_count', 'created_at', 'favourites_count', 'utc_offset', 'time_zone', 'geo_enabled', 'verified', 'statuses_count', 'lang', 'status', 'contributors_enabled', 'is_translator', 'is_translation_enabled', 'profile_background_color', 'profile_background_image_url', 'profile_background_image_url_https', 'profile_background_tile', 'profile_image_url', 'profile_image_url_https', 'profile_banner_url', 'profile_link_color', 'profile_sidebar_border_color', 'profile_sidebar_fill_color', 'profile_text_color', 'profile_use_background_image', 'has_extended_profile', 'default_profile', 'default_profile_image', 'following', 'follow_request_sent', 'notifications', 'translator_type', 'withheld_in_countries'])

## Get User Details

In [14]:
# Define DataFrame columns for all the User data elements we want to collect

govs_d_user_df = pd.DataFrame(columns=['screen_name',
                                    'id',
                                    'name',
                                    'location',
                                    'description',
                                    'url',
                                    'created_at',
                                    'verified',
                                    'protected',
                                    # 'followers',
                                    'followers_count',
                                    # 'followers_id',
                                    # 'following',
                                    'follow_request_sent',
                                    # 'follow',
                                    # 'friends',
                                    'friends_count',
                                    'favorites_count',
                                    'listed_count',
                                    # 'status',
                                    # 'statuses_count',
                                    ])

govs_r_user_df = pd.DataFrame(columns=['screen_name',
                                    'id',
                                    'name',
                                    'location',
                                    'description',
                                    'url',
                                    'created_at',
                                    'verified',
                                    'protected',
                                    # 'followers',
                                    'followers_count',
                                    # 'followers_id',
                                    # 'following',
                                    'follow_request_sent',
                                    # 'follow',
                                    # 'friends',
                                    'friends_count',
                                    'favorites_count',
                                    'listed_count',
                                    # 'status',
                                    # 'statuses_count',
                                    ])

In [15]:
govs_d_df.head()

,state,democrat,screen_name
0,wisconsin,https://twitter.com/GovEvers,GovEvers
1,orgeon,https://twitter.com/TinaKotek,TinaKotek
2,nevada,https://twitter.com/GovSisolak,GovSisolak
3,arizona,https://twitter.com/katiehobbs,katiehobbs
4,kansas,https://twitter.com/LauraKellyKS,LauraKellyKS


In [16]:
# Get User data for each Dem Governor

for i,agov_username in enumerate(govs_d_df.screen_name.to_list()):
    print(f'Processing #{i}: {agov_username}')
    try:
        # https://docs.tweepy.org/en/v4.8.0/api.html#tweepy.API.get_user
        tweepy_user_obj = api.get_user(screen_name=agov_username)
        print(f'    Twitter ID: {agov_id}')
        
        agov_user_info_ls = [tweepy_user_obj.screen_name,
                             tweepy_user_obj.id,
                             tweepy_user_obj.name,
                             tweepy_user_obj.location,
                             tweepy_user_obj.description,
                             tweepy_user_obj.url,
                             tweepy_user_obj.created_at,
                             tweepy_user_obj.verified,
                             tweepy_user_obj.protected,
                             # tweepy_user_obj.followers,
                             tweepy_user_obj.followers_count,
                             # tweepy_user_obj.followers_ids,
                             # tweepy_user_obj.following,
                             tweepy_user_obj.follow_request_sent,
                             # tweepy_user_obj.follow,
                             # tweepy_user_obj.friends,
                             tweepy_user_obj.friends_count,
                             tweepy_user_obj.favourites_count,
                             tweepy_user_obj.listed_count,
                             # tweepy_user_obj.status,
                             # tweepy_user_obj.statuses_count
                             ]  
        print(f'    Appending row: {agov_user_info_ls}\n\n')
        govs_d_user_df.loc[len(govs_d_user_df)] = agov_user_info_ls                

        
    except:
        print(f'    ERROR: Could not retrieve Tweepy User Object')


Processing #0: GovEvers
    ERROR: Could not retrieve Tweepy User Object
Processing #1: TinaKotek
    ERROR: Could not retrieve Tweepy User Object
Processing #2: GovSisolak
    ERROR: Could not retrieve Tweepy User Object
Processing #3: katiehobbs
    ERROR: Could not retrieve Tweepy User Object
Processing #4: LauraKellyKS
    ERROR: Could not retrieve Tweepy User Object
Processing #5: GovMLG
    ERROR: Could not retrieve Tweepy User Object
Processing #6: gretchenwhitmer
    ERROR: Could not retrieve Tweepy User Object
Processing #7: GovJanetMills
    ERROR: Could not retrieve Tweepy User Object
Processing #8: staceyabrams
    ERROR: Could not retrieve Tweepy User Object
Processing #9: GovTimWalz
    ERROR: Could not retrieve Tweepy User Object
Processing #10: JoshShapiroPA
    ERROR: Could not retrieve Tweepy User Object
Processing #11: joy4ok
    ERROR: Could not retrieve Tweepy User Object
Processing #12: CharlieCrist
    ERROR: Could not retrieve Tweepy User Object
Processing #13: 

In [793]:
# Verfiy Dem User data
 
govs_d_user_df.head(3)
govs_d_user_df.info()

,screen_name,id,name,location,description,url,created_at,verified,protected,followers_count,follow_request_sent,friends_count,favorites_count,listed_count
0,GovEvers,234896532,Governor Tony Evers,"Madison, Wisconsin","Husband to Kathy, lifelong educator, Euchre an...",https://t.co/JyH2JWfeyt,2011-01-06 21:05:10+00:00,True,False,103344,False,246,167,900
1,TinaKotek,2835931010,Tina Kotek,Oregon,Progressive Democrat. Candidate for Oregon Gov...,https://t.co/U0kpZIDyUZ,2014-09-30 18:28:02+00:00,True,False,20042,False,338,1510,289
2,GovSisolak,1080586433088258048,Governor Sisolak,"Nevada, USA",Governor of Nevada. Proud Dad and Husband. Sta...,https://t.co/9iq0pinVZs,2019-01-02 22:07:36+00:00,True,False,100250,False,123,1987,602


<class 'pandas.core.frame.DataFrame'>
Int64Index: 35 entries, 0 to 34
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   screen_name          35 non-null     object             
 1   id                   35 non-null     int64              
 2   name                 35 non-null     object             
 3   location             35 non-null     object             
 4   description          35 non-null     object             
 5   url                  35 non-null     object             
 6   created_at           35 non-null     datetime64[ns, UTC]
 7   verified             35 non-null     bool               
 8   protected            35 non-null     bool               
 9   followers_count      35 non-null     int64              
 10  follow_request_sent  35 non-null     bool               
 11  friends_count        35 non-null     int64              
 12  favorites_count      35 

In [794]:
# Get User data for each Rep Governor

for i,agov_username in enumerate(govs_r_df.screen_name.to_list()):
    print(f'Processing #{i}: {agov_username}')
    try:
        # https://docs.tweepy.org/en/v4.8.0/api.html#tweepy.API.get_user
        tweepy_user_obj = api.get_user(screen_name=agov_username)
        print(f'    Twitter ID: {agov_id}')
        
        agov_user_info_ls = [tweepy_user_obj.screen_name,
                             tweepy_user_obj.id,
                             tweepy_user_obj.name,
                             tweepy_user_obj.location,
                             tweepy_user_obj.description,
                             tweepy_user_obj.url,
                             tweepy_user_obj.created_at,
                             tweepy_user_obj.verified,
                             tweepy_user_obj.protected,
                             # tweepy_user_obj.followers,
                             tweepy_user_obj.followers_count,
                             # tweepy_user_obj.followers_ids,
                             # tweepy_user_obj.following,
                             tweepy_user_obj.follow_request_sent,
                             # tweepy_user_obj.follow,
                             # tweepy_user_obj.friends,
                             tweepy_user_obj.friends_count,
                             tweepy_user_obj.favourites_count,
                             tweepy_user_obj.listed_count,
                             # tweepy_user_obj.status,
                             # tweepy_user_obj.statuses_count
                             ]  
        print(f'    Appending row: {agov_user_info_ls}\n\n')
        govs_r_user_df.loc[len(govs_r_user_df)] = agov_user_info_ls                
        
    except:
        print(f'    ERROR: Could not retrieve Tweepy User Object')


Processing #0: michelsforgov
    Twitter ID: 1401246913542836224
    Appending row: ['michelsforgov', 1514274154274512901, 'Tim Michels', 'Wisconsin', 'Army Veteran. Successful Businessman. Outsider.\n\nTim Michels is running for governor to get Wisconsin back on the right track.\n\nJoin the team!\nhttps://t.co/lHKqQnW72j', 'https://t.co/6hqLqdIXZh', datetime.datetime(2022, 4, 13, 16, 8, tzinfo=datetime.timezone.utc), True, False, 11839, False, 165, 215, 90]


Processing #1: ChristineDrazan
    Twitter ID: 1401246913542836224
    Appending row: ['ChristineDrazan', 965763444627550210, 'Christine Drazan', 'Oregon, USA', 'Republican candidate for governor. I want to fix our schools, keep our communities safe, and make Oregon more affordable. Ready to lead #OnDay1.', 'https://t.co/NXGQ3ZaQWN', datetime.datetime(2018, 2, 20, 1, 42, 2, tzinfo=datetime.timezone.utc), True, False, 18863, False, 343, 652, 102]


Processing #2: JoeLombardoNV
    Twitter ID: 1401246913542836224
    Appending row:

In [795]:
# Verfiy Rep User data
 
govs_r_user_df.head(3)
govs_r_user_df.info()

,screen_name,id,name,location,description,url,created_at,verified,protected,followers_count,follow_request_sent,friends_count,favorites_count,listed_count
0,michelsforgov,1514274154274512901,Tim Michels,Wisconsin,Army Veteran. Successful Businessman. Outsider...,https://t.co/6hqLqdIXZh,2022-04-13 16:08:00+00:00,True,False,11839,False,165,215,90
1,ChristineDrazan,965763444627550210,Christine Drazan,"Oregon, USA",Republican candidate for governor. I want to f...,https://t.co/NXGQ3ZaQWN,2018-02-20 01:42:02+00:00,True,False,18863,False,343,652,102
2,JoeLombardoNV,1389345377137176579,Joe Lombardo,"Nevada, USA",Army Veteran. Law Enforcement Officer of 34 ye...,https://t.co/SLKRLjxdld,2021-05-03 22:25:37+00:00,True,False,8275,False,35,168,73


<class 'pandas.core.frame.DataFrame'>
Int64Index: 35 entries, 0 to 34
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   screen_name          35 non-null     object             
 1   id                   35 non-null     int64              
 2   name                 35 non-null     object             
 3   location             35 non-null     object             
 4   description          35 non-null     object             
 5   url                  34 non-null     object             
 6   created_at           35 non-null     datetime64[ns, UTC]
 7   verified             35 non-null     bool               
 8   protected            35 non-null     bool               
 9   followers_count      35 non-null     int64              
 10  follow_request_sent  35 non-null     bool               
 11  friends_count        35 non-null     int64              
 12  favorites_count      35 

In [797]:
%whos DataFrame

Variable             Type         Data/Info
-------------------------------------------
df                   DataFrame      edit_history_tweet_ids <...>..  1590272396920954880  
gov_user_df          DataFrame            screen_name      <...>588  \n34            27  
govs_d_df            DataFrame                 state       <...>dtforgov      heidtforgov
govs_d_timeline_df   DataFrame    Empty DataFrame\nColumns:<...>\n\n[0 rows x 26 columns]
govs_d_user_df       DataFrame            screen_name      <...> 27  \n34            12  
govs_df              DataFrame                 state       <...>twitter.com/heidtforgov  
govs_r_df            DataFrame                 state       <...>alABundy       RealABundy
govs_r_user_df       DataFrame            screen_name      <...>147  \n34            19  
govs_tweets_df       DataFrame    Empty DataFrame\nColumns: []\nIndex: []
govs_user_df         DataFrame            screen_name      <...>588  \n34            27  
my_followers_df      DataFra

## Join DataFrames

In [17]:
govs_d_df.head()

,state,democrat,screen_name
0,wisconsin,https://twitter.com/GovEvers,GovEvers
1,orgeon,https://twitter.com/TinaKotek,TinaKotek
2,nevada,https://twitter.com/GovSisolak,GovSisolak
3,arizona,https://twitter.com/katiehobbs,katiehobbs
4,kansas,https://twitter.com/LauraKellyKS,LauraKellyKS


In [799]:
# Joint 2 Dem DataFrames: (1) original gov_df and (2) gov_user_df 

govs_d_user_df = pd.merge(govs_d_df, govs_d_user_df, on='screen_name') 
govs_d_user_df.head()
govs_d_user_df.info()

,state,democrat,screen_name,id,name,location,description,url,created_at,verified,protected,followers_count,follow_request_sent,friends_count,favorites_count,listed_count
0,wisconsin,https://twitter.com/GovEvers,GovEvers,234896532,Governor Tony Evers,"Madison, Wisconsin","Husband to Kathy, lifelong educator, Euchre an...",https://t.co/JyH2JWfeyt,2011-01-06 21:05:10+00:00,True,False,103344,False,246,167,900
1,orgeon,https://twitter.com/TinaKotek,TinaKotek,2835931010,Tina Kotek,Oregon,Progressive Democrat. Candidate for Oregon Gov...,https://t.co/U0kpZIDyUZ,2014-09-30 18:28:02+00:00,True,False,20042,False,338,1510,289
2,nevada,https://twitter.com/GovSisolak,GovSisolak,1080586433088258048,Governor Sisolak,"Nevada, USA",Governor of Nevada. Proud Dad and Husband. Sta...,https://t.co/9iq0pinVZs,2019-01-02 22:07:36+00:00,True,False,100250,False,123,1987,602
3,arizona,https://twitter.com/katiehobbs,katiehobbs,26659619,Katie Hobbs,"Phoenix, AZ","Social worker, mom, wife, Arizona’s Secretary ...",https://t.co/otdUvzR8kj,2009-03-26 01:55:21+00:00,True,False,135381,False,1576,6488,868
4,kansas,https://twitter.com/LauraKellyKS,LauraKellyKS,364864453,Laura Kelly,"Topeka, KS","48th Governor of Kansas, Wife, Mother, Champio...",https://t.co/ErZXA4bB3F,2011-08-30 13:39:23+00:00,True,False,55018,False,553,3634,390


<class 'pandas.core.frame.DataFrame'>
Int64Index: 35 entries, 0 to 34
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   state                35 non-null     object             
 1   democrat             35 non-null     object             
 2   screen_name          35 non-null     object             
 3   id                   35 non-null     int64              
 4   name                 35 non-null     object             
 5   location             35 non-null     object             
 6   description          35 non-null     object             
 7   url                  35 non-null     object             
 8   created_at           35 non-null     datetime64[ns, UTC]
 9   verified             35 non-null     bool               
 10  protected            35 non-null     bool               
 11  followers_count      35 non-null     int64              
 12  follow_request_sent  35 

In [800]:
# Save Dem User data 

filename_out = 'twitter_2022midterm_us_governors_d_user_data.csv'
 
govs_d_user_df.to_csv(filename_out, index=False)
govs_d_user_df.head()

,state,democrat,screen_name,id,name,location,description,url,created_at,verified,protected,followers_count,follow_request_sent,friends_count,favorites_count,listed_count
0,wisconsin,https://twitter.com/GovEvers,GovEvers,234896532,Governor Tony Evers,"Madison, Wisconsin","Husband to Kathy, lifelong educator, Euchre an...",https://t.co/JyH2JWfeyt,2011-01-06 21:05:10+00:00,True,False,103344,False,246,167,900
1,orgeon,https://twitter.com/TinaKotek,TinaKotek,2835931010,Tina Kotek,Oregon,Progressive Democrat. Candidate for Oregon Gov...,https://t.co/U0kpZIDyUZ,2014-09-30 18:28:02+00:00,True,False,20042,False,338,1510,289
2,nevada,https://twitter.com/GovSisolak,GovSisolak,1080586433088258048,Governor Sisolak,"Nevada, USA",Governor of Nevada. Proud Dad and Husband. Sta...,https://t.co/9iq0pinVZs,2019-01-02 22:07:36+00:00,True,False,100250,False,123,1987,602
3,arizona,https://twitter.com/katiehobbs,katiehobbs,26659619,Katie Hobbs,"Phoenix, AZ","Social worker, mom, wife, Arizona’s Secretary ...",https://t.co/otdUvzR8kj,2009-03-26 01:55:21+00:00,True,False,135381,False,1576,6488,868
4,kansas,https://twitter.com/LauraKellyKS,LauraKellyKS,364864453,Laura Kelly,"Topeka, KS","48th Governor of Kansas, Wife, Mother, Champio...",https://t.co/ErZXA4bB3F,2011-08-30 13:39:23+00:00,True,False,55018,False,553,3634,390


In [801]:
# Joint 2 Rep DataFrames: (1) original gov_df and (2) gov_user_df 

govs_r_user_df = pd.merge(govs_r_df, govs_r_user_df, on='screen_name') 
govs_r_user_df.head()
govs_r_user_df.info()

,state,republican,screen_name,id,name,location,description,url,created_at,verified,protected,followers_count,follow_request_sent,friends_count,favorites_count,listed_count
0,wisconsin,https://twitter.com/michelsforgov,michelsforgov,1514274154274512901,Tim Michels,Wisconsin,Army Veteran. Successful Businessman. Outsider...,https://t.co/6hqLqdIXZh,2022-04-13 16:08:00+00:00,True,False,11839,False,165,215,90
1,orgeon,https://twitter.com/ChristineDrazan,ChristineDrazan,965763444627550210,Christine Drazan,"Oregon, USA",Republican candidate for governor. I want to f...,https://t.co/NXGQ3ZaQWN,2018-02-20 01:42:02+00:00,True,False,18863,False,343,652,102
2,nevada,https://twitter.com/JoeLombardoNV,JoeLombardoNV,1389345377137176579,Joe Lombardo,"Nevada, USA",Army Veteran. Law Enforcement Officer of 34 ye...,https://t.co/SLKRLjxdld,2021-05-03 22:25:37+00:00,True,False,8275,False,35,168,73
3,arizona,https://twitter.com/KariLake,KariLake,1082197856,Kari Lake,Iowa & Arizona,Official Twitter Account for Trump-Endorsed Ca...,https://t.co/EmzgORJKeQ,2013-01-12 07:47:51+00:00,True,False,585897,False,981,24421,1122
4,kansas,https://twitter.com/DerekSchmidtKS,DerekSchmidtKS,161138443,Derek Schmidt,,Trusted Leader. Proven Winner. Kansas Conserva...,https://t.co/ZYJ5mUrVYw,2010-06-30 02:39:27+00:00,True,False,4884,False,17,10,61


<class 'pandas.core.frame.DataFrame'>
Int64Index: 35 entries, 0 to 34
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   state                35 non-null     object             
 1   republican           35 non-null     object             
 2   screen_name          35 non-null     object             
 3   id                   35 non-null     int64              
 4   name                 35 non-null     object             
 5   location             35 non-null     object             
 6   description          35 non-null     object             
 7   url                  34 non-null     object             
 8   created_at           35 non-null     datetime64[ns, UTC]
 9   verified             35 non-null     bool               
 10  protected            35 non-null     bool               
 11  followers_count      35 non-null     int64              
 12  follow_request_sent  35 

In [488]:
# Save Rep User data 

filename_out = 'twitter_2022midterm_us_governors_r_user_data.csv'
 
govs_r_user_df.to_csv(filename_out, index=False)
govs_r_user_df.head()

,state,republican,screen_name,id,name,location,description,url,created_at,verified,protected,followers_count,follow_request_sent,friends_count,favorites_count,listed_count
0,wisconsin,https://twitter.com/michelsforgov,michelsforgov,1514274154274512901,Tim Michels,Wisconsin,Army Veteran. Successful Businessman. Outsider...,https://t.co/6hqLqdIXZh,2022-04-13 16:08:00+00:00,True,False,11855,False,165,215,92
1,orgeon,https://twitter.com/ChristineDrazan,ChristineDrazan,965763444627550210,Christine Drazan,"Oregon, USA",Republican candidate for governor. I want to f...,https://t.co/NXGQ3ZaQWN,2018-02-20 01:42:02+00:00,True,False,18707,False,345,652,104
2,nevada,https://twitter.com/JoeLombardoNV,JoeLombardoNV,1389345377137176579,Joe Lombardo,"Nevada, USA",Army Veteran. Law Enforcement Officer of 34 ye...,https://t.co/SLKRLjxdld,2021-05-03 22:25:37+00:00,True,False,8091,False,35,168,76
3,arizona,https://twitter.com/KariLake,KariLake,1082197856,Kari Lake,Iowa & Arizona,Official Twitter Account for Trump-Endorsed Ca...,https://t.co/EmzgORJKeQ,2013-01-12 07:47:51+00:00,True,False,572139,False,982,24404,1126
4,kansas,https://twitter.com/DerekSchmidtKS,DerekSchmidtKS,161138443,Derek Schmidt,,Trusted Leader. Proven Winner. Kansas Conserva...,https://t.co/ZYJ5mUrVYw,2010-06-30 02:39:27+00:00,True,False,4879,False,17,10,62


## Get User Timelines

In [489]:
%whos DataFrame

Variable             Type         Data/Info
-------------------------------------------
df                   DataFrame                             <...>forgov      heidtforgov  
gov_user_df          DataFrame            screen_name      <...>588  \n34            27  
govs_d_df            DataFrame                 state       <...>dtforgov      heidtforgov
govs_d_timeline_df   DataFrame    Empty DataFrame\nColumns:<...>\n\n[0 rows x 26 columns]
govs_d_user_df       DataFrame                 state       <...> 28  \n34            13  
govs_df              DataFrame                 state       <...>twitter.com/heidtforgov  
govs_r_df            DataFrame                 state       <...>alABundy       RealABundy
govs_r_user_df       DataFrame                 state       <...>148  \n34            19  
govs_tweets_df       DataFrame    Empty DataFrame\nColumns: []\nIndex: []
govs_user_df         DataFrame            screen_name      <...>588  \n34            27  
temp_df              DataFra

In [494]:
# Test

ascreen_name = 'KariLake'
user_timeline_tweets = api.user_timeline(screen_name=ascreen_name)
type(user_timeline_tweets)
len(user_timeline_tweets)

tweepy.models.ResultSet

20

In [495]:
user_timeline_tweets[0]._json

{'created_at': 'Wed Nov 09 05:49:56 +0000 2022',
 'id': 1590220119862493185,
 'id_str': '1590220119862493185',
 'text': '🚨 ELECTION UPDATE 🚨 \n\nBrand New ELECTION DAY vote from Yuma &amp; Greenlee is Massive for Kari Lake \n\n@KariLake - 76.4%… https://t.co/bQwt1JErcs',
 'truncated': True,
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [{'screen_name': 'KariLake',
    'name': 'Kari Lake',
    'id': 1082197856,
    'id_str': '1082197856',
    'indices': [102, 111]}],
  'urls': [{'url': 'https://t.co/bQwt1JErcs',
    'expanded_url': 'https://twitter.com/i/web/status/1590220119862493185',
    'display_url': 'twitter.com/i/web/status/1…',
    'indices': [121, 144]}]},
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 1082197856,
  'id_str': '108

In [496]:
# Get list of dict.keys in Timeline Tweets object

timeline_tweet_keys_ls = list(user_timeline_tweets[0]._json.keys())
timeline_tweet_keys_ls

['created_at',
 'id',
 'id_str',
 'text',
 'truncated',
 'entities',
 'source',
 'in_reply_to_status_id',
 'in_reply_to_status_id_str',
 'in_reply_to_user_id',
 'in_reply_to_user_id_str',
 'in_reply_to_screen_name',
 'user',
 'geo',
 'coordinates',
 'place',
 'contributors',
 'is_quote_status',
 'retweet_count',
 'favorite_count',
 'favorited',
 'retweeted',
 'possibly_sensitive',
 'lang']

In [497]:
indx = 5

pprint(user_timeline_tweets[indx]._json)

{'contributors': None,
 'coordinates': None,
 'created_at': 'Wed Nov 09 04:50:16 +0000 2022',
 'entities': {'hashtags': [],
              'symbols': [],
              'urls': [{'display_url': 'twitter.com/i/web/status/1…',
                        'expanded_url': 'https://twitter.com/i/web/status/1590205106313912322',
                        'indices': [121, 144],
                        'url': 'https://t.co/JKNRuhfPzO'}],
              'user_mentions': [{'id': 1082197856,
                                 'id_str': '1082197856',
                                 'indices': [80, 89],
                                 'name': 'Kari Lake',
                                 'screen_name': 'KariLake'}]},
 'favorite_count': 17083,
 'favorited': False,
 'geo': None,
 'id': 1590205106313912322,
 'id_str': '1590205106313912322',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_q

In [498]:
indx=3

user_timeline_tweets[indx]._json['text']
user_timeline_tweets[indx]._json['retweet_count']

'WATCH: @KariLake speaks at Election Night Party \n\nhttps://t.co/BqJ67SFdjY'

272

In [499]:
%whos DataFrame

Variable             Type         Data/Info
-------------------------------------------
df                   DataFrame                             <...>forgov      heidtforgov  
gov_user_df          DataFrame            screen_name      <...>588  \n34            27  
govs_d_df            DataFrame                 state       <...>dtforgov      heidtforgov
govs_d_timeline_df   DataFrame    Empty DataFrame\nColumns:<...>\n\n[0 rows x 26 columns]
govs_d_user_df       DataFrame                 state       <...> 28  \n34            13  
govs_df              DataFrame                 state       <...>twitter.com/heidtforgov  
govs_r_df            DataFrame                 state       <...>alABundy       RealABundy
govs_r_user_df       DataFrame                 state       <...>148  \n34            19  
govs_tweets_df       DataFrame    Empty DataFrame\nColumns: []\nIndex: []
govs_user_df         DataFrame            screen_name      <...>588  \n34            27  
temp_df              DataFra

In [500]:
type(user_timeline_tweets[indx]._json['possibly_sensitive'])

bool

In [501]:
# Define global timelime of tweets for both Dem/Rep governors

govs_d_timeline_df = pd.DataFrame({'screen_name': pd.Series(dtype='str'),
                                   'created_at': pd.Series(dtype='str'),  # datetime
                                   'id': pd.Series(dtype='int'),
                                   'id_str': pd.Series(dtype='str'),
                                   'text': pd.Series(dtype='str'),
                                   'truncated': pd.Series(dtype='bool'),
                                   'entities': pd.Series(dtype='str'),  # dict
                                   'extended_entities': pd.Series(dtype='str'),
                                   'source': pd.Series(dtype='str'),
                                   'in_reply_to_status_id': pd.Series(dtype='int'),
                                   'in_reply_to_status_id_str': pd.Series(dtype='str'),
                                   'in_reply_to_user_id': pd.Series(dtype='int'),
                                   'in_reply_to_user_id_str': pd.Series(dtype='str'),
                                   'in_reply_to_screen_name': pd.Series(dtype='str'),
                                   'user': pd.Series(dtype='str'),   # dict
                                   'geo': pd.Series(dtype='str'),
                                   'coordinates': pd.Series(dtype='str'),
                                   'place': pd.Series(dtype='str'),
                                   'contributors': pd.Series(dtype='str'),
                                   'is_quote_status': pd.Series(dtype='bool'),
                                   'retweet_count': pd.Series(dtype='int'),
                                   'favorite_count': pd.Series(dtype='int'),
                                   'favorited': pd.Series(dtype='bool'),
                                   'retweeted': pd.Series(dtype='bool'),
                                   'possibly_sensitive': pd.Series(dtype='str'),
                                   'lang': pd.Series(dtype='str')
                                   })
govs_d_timeline_df.head()

,screen_name,created_at,id,id_str,text,truncated,entities,extended_entities,source,in_reply_to_status_id,...,coordinates,place,contributors,is_quote_status,retweet_count,favorite_count,favorited,retweeted,possibly_sensitive,lang


In [502]:
# CAUTION: Long-running process, normal OAuth can only retrieve 900/15min rate

def get_user_tweets(username, govs_timeline_df):
    # global api, tweets_govs_df,count,error
    error = 0
    try:
        # https://docs.tweepy.org/en/stable/api.html#tweepy.API.user_timeline
        user_timeline_tweets_res = api.user_timeline(screen_name=username,
                                                     count=50,
                                                     include_rts = False,
                                                     tweet_mode = 'extended')
        
        print(f'Retrieved #{len(user_timeline_tweets_res)} Tweets in {username} Timeline')
        for i,atweet in enumerate(user_timeline_tweets_res):
            print(f'Processing Tweet #{i}...\n')
            atweet_ls = [username]
            for j,akey in enumerate(timeline_tweet_keys_ls):
                try:
                    avalue = atweet._json[akey]
                    # avalue = user_timeline_tweets_res[i]._json[akey]
                    print(f'    {akey} : {avalue}')
                    if avalue:
                        atweet_ls.append(avalue)
                    else:
                        atweet_ls.append('')
                    print(f'  Appending atweet_ls: {atweet_ls}\n\n')
                except:
                    print(f'    {akey} : None')
                    atweet_ls.append('')
                    continue
            df_cols = ['screen_name'] + timeline_tweet_keys_ls
            df = pd.DataFrame(columns=df_cols)
            df.append(atweet_ls)

            # df_copy = df.copy(deep=True)
            govs_timeline_df = govs_timeline_df.append(df) # df_copy)

    except:
        error+=1
        print(error, count)
        if count%100==True:
            print(count)

In [503]:
# Test

get_user_tweets('GovEvers', govs_d_timeline_df)

Retrieved #41 Tweets in GovEvers Timeline
Processing Tweet #0...

    created_at : Wed Nov 09 00:08:56 +0000 2022
  Appending atweet_ls: ['GovEvers', 'Wed Nov 09 00:08:56 +0000 2022']


    id : 1590134306701471746
  Appending atweet_ls: ['GovEvers', 'Wed Nov 09 00:08:56 +0000 2022', 1590134306701471746]


    id_str : 1590134306701471746
  Appending atweet_ls: ['GovEvers', 'Wed Nov 09 00:08:56 +0000 2022', 1590134306701471746, '1590134306701471746']


    text : None
    truncated : False
  Appending atweet_ls: ['GovEvers', 'Wed Nov 09 00:08:56 +0000 2022', 1590134306701471746, '1590134306701471746', '', '']


    entities : {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/DwTqF7XfTy', 'expanded_url': 'https://twitter.com/govevers/status/1590042048476086273', 'display_url': 'twitter.com/govevers/statu…', 'indices': [196, 219]}]}
  Appending atweet_ls: ['GovEvers', 'Wed Nov 09 00:08:56 +0000 2022', 1590134306701471746, '1590134306701471746', '', '', {

C:\Users\jonch\AppData\Local\Temp\ipykernel_9268\215947457.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(atweet_ls)
C:\Users\jonch\AppData\Local\Temp\ipykernel_9268\215947457.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  govs_timeline_df = govs_timeline_df.append(df) # df_copy)
C:\Users\jonch\AppData\Local\Temp\ipykernel_9268\215947457.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append(atweet_ls)
C:\Users\jonch\AppData\Local\Temp\ipykernel_9268\215947457.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  govs_timeline_df = govs_timeline_df.append(df) # df_copy)
C:\Users\jonch\AppData\Local\Temp\ipyk

In [504]:
govs_d_timeline_df.head()
govs_d_timeline_df.info()

,screen_name,created_at,id,id_str,text,truncated,entities,extended_entities,source,in_reply_to_status_id,...,coordinates,place,contributors,is_quote_status,retweet_count,favorite_count,favorited,retweeted,possibly_sensitive,lang


<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   screen_name                0 non-null      object
 1   created_at                 0 non-null      object
 2   id                         0 non-null      int32 
 3   id_str                     0 non-null      object
 4   text                       0 non-null      object
 5   truncated                  0 non-null      bool  
 6   entities                   0 non-null      object
 7   extended_entities          0 non-null      object
 8   source                     0 non-null      object
 9   in_reply_to_status_id      0 non-null      int32 
 10  in_reply_to_status_id_str  0 non-null      object
 11  in_reply_to_user_id        0 non-null      int32 
 12  in_reply_to_user_id_str    0 non-null      object
 13  in_reply_to_screen_name    0 non-null      object
 14  user                       

In [253]:
for i,irow in govs_d_user_df.iterrows():
    print(f'Row #{i}: {irow}')

Row #0: state                                                          wisconsin
democrat                                    https://twitter.com/GovEvers
screen_name                                                     GovEvers
id                                                             234896532
name                                                 Governor Tony Evers
location                                              Madison, Wisconsin
description            Husband to Kathy, lifelong educator, Euchre an...
url                                              https://t.co/JyH2JWfeyt
created_at                                     2011-01-06 21:05:10+00:00
verified                                                            True
protected                                                          False
followers_count                                                   102207
follow_request_sent                                                False
friends_count                              

# [SKIP] Option 2: Tweepy: v2.0 API

## OAuth 2.0

In [802]:
# Regular
client = tweepy.Client(bearer_token)

# Get Response obj in Requests, able to translate into JSON
import requests
import json
client1 = tweepy.Client(bearer_token = bearer_token, return_type = requests.Response)
client1


## Test

In [803]:
# Test
 
user_name_obj = client.get_user(username='KariLake', user_fields=['created_at'])
# type(user_name_obj)
# len(user_name_obj)
# dir(user_name_obj.data)
user_id = user_name_obj.data.id
user_id
user_name_obj.data.name
user_name_obj.data.username
user_name_obj.includes

1082197856

'Kari Lake'

'KariLake'

{}

In [578]:
%whos DataFrame

Variable             Type         Data/Info
-------------------------------------------
df                   DataFrame                             <...>forgov      heidtforgov  
gov_user_df          DataFrame            screen_name      <...>588  \n34            27  
govs_d_df            DataFrame                 state       <...>dtforgov      heidtforgov
govs_d_timeline_df   DataFrame    Empty DataFrame\nColumns:<...>\n\n[0 rows x 26 columns]
govs_d_user_df       DataFrame                 state       <...> 28  \n34            13  
govs_df              DataFrame                 state       <...>twitter.com/heidtforgov  
govs_r_df            DataFrame                 state       <...>alABundy       RealABundy
govs_r_user_df       DataFrame                 state       <...>148  \n34            19  
govs_tweets_df       DataFrame    Empty DataFrame\nColumns: []\nIndex: []
govs_user_df         DataFrame            screen_name      <...>588  \n34            27  
temp_df              DataFra

In [580]:
govs_d_df.head()

,state,democrat,screen_name
0,wisconsin,https://twitter.com/GovEvers,GovEvers
1,orgeon,https://twitter.com/TinaKotek,TinaKotek
2,nevada,https://twitter.com/GovSisolak,GovSisolak
3,arizona,https://twitter.com/katiehobbs,katiehobbs
4,kansas,https://twitter.com/LauraKellyKS,LauraKellyKS


In [574]:
# https://docs.tweepy.org/en/v4.8.0/client.html#id14
# https://developer.twitter.com/en/docs/twitter-api/data-dictionary/object-model/user
 
# user_name_obj = client.get_user(id=1082197856, user_fields=['created_at'])
user_name_obj = client.get_user(username='KariLake', expansions=['pinned_tweet_id'], user_fields=['created_at','description'])
user_name_dt = user_name_obj._asdict()
pprint(user_name_dt)
user_name_obj.data.keys()
# (user_name_obj.data)
# print(user_name_obj.data)
# dir(users_name_obj.index)
user_name_obj.includes

{'data': <User id=1082197856 name=Kari Lake username=KariLake>,
 'errors': [],
 'includes': {'tweets': [<Tweet id=1575893306487103498 text="I’ve shared a lot of stories with you over the last few decades, but I've never shared mine. \n\nJoin me on my journey:\nhttps://t.co/XJKyq7uzXn\n\nPlease share this video! https://t.co/doE6BXex2K">]},
 'meta': {}}


KeysView(<User id=1082197856 name=Kari Lake username=KariLake>)

{'tweets': [<Tweet id=1575893306487103498 text="I’ve shared a lot of stories with you over the last few decades, but I've never shared mine. \n\nJoin me on my journey:\nhttps://t.co/XJKyq7uzXn\n\nPlease share this video! https://t.co/doE6BXex2K">]}

In [398]:
my_query = 'midjourney prompt -is:retweet has:media'

response = client.search_recent_tweets(query=my_query, tweet_fields=['created_at','lang'], expansions=['author_id'])

print('hello')

hello


In [399]:
len(response)

4

In [400]:
for tweet in response.data:
    print(tweet.id)

1590205976321613826
1590205773262761984
1590191064454901760
1590188601295659010
1590177362938847232
1590166978269831168
1590150957215543296
1590149389753798656
1590147558789160960
1590141349164355584


In [401]:
print(dir(response.data))

['__add__', '__class__', '__class_getitem__', '__contains__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__iadd__', '__imul__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__rmul__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', 'append', 'clear', 'copy', 'count', 'extend', 'index', 'insert', 'pop', 'remove', 'reverse', 'sort']


## Get User ID

In [585]:
%whos DataFrame

Variable             Type         Data/Info
-------------------------------------------
df                   DataFrame                             <...>forgov      heidtforgov  
gov_user_df          DataFrame            screen_name      <...>588  \n34            27  
govs_d_df            DataFrame                 state       <...>34  1513378188168101897  
govs_d_timeline_df   DataFrame    Empty DataFrame\nColumns:<...>\n\n[0 rows x 26 columns]
govs_d_user_df       DataFrame                 state       <...> 28  \n34            13  
govs_df              DataFrame                 state       <...>twitter.com/heidtforgov  
govs_r_df            DataFrame                 state       <...>34  1401246913542836224  
govs_r_user_df       DataFrame                 state       <...>148  \n34            19  
govs_tweets_df       DataFrame    Empty DataFrame\nColumns: []\nIndex: []
govs_user_df         DataFrame            screen_name      <...>588  \n34            27  
temp_df              DataFra

In [586]:
govs_d_df.head()

,state,democrat,screen_name,id
0,wisconsin,https://twitter.com/GovEvers,GovEvers,234896532
1,orgeon,https://twitter.com/TinaKotek,TinaKotek,2835931010
2,nevada,https://twitter.com/GovSisolak,GovSisolak,1080586433088258048
3,arizona,https://twitter.com/katiehobbs,katiehobbs,26659619
4,kansas,https://twitter.com/LauraKellyKS,LauraKellyKS,364864453


In [644]:
govs_d_df['id'] = govs_d_df['screen_name'].apply(lambda x: client.get_user(username=x).data.id)
govs_d_df.head()

,state,democrat,screen_name,id
0,wisconsin,https://twitter.com/GovEvers,GovEvers,234896532
1,orgeon,https://twitter.com/TinaKotek,TinaKotek,2835931010
2,nevada,https://twitter.com/GovSisolak,GovSisolak,1080586433088258048
3,arizona,https://twitter.com/katiehobbs,katiehobbs,26659619
4,kansas,https://twitter.com/LauraKellyKS,LauraKellyKS,364864453


In [583]:
govs_r_df.head()

,state,republican,screen_name
0,wisconsin,https://twitter.com/michelsforgov,michelsforgov
1,orgeon,https://twitter.com/ChristineDrazan,ChristineDrazan
2,nevada,https://twitter.com/JoeLombardoNV,JoeLombardoNV
3,arizona,https://twitter.com/KariLake,KariLake
4,kansas,https://twitter.com/DerekSchmidtKS,DerekSchmidtKS


In [584]:
govs_r_df['id'] = govs_r_df['screen_name'].apply(lambda x: client.get_user(username=x).data.id)
govs_r_df.head()

,state,republican,screen_name,id
0,wisconsin,https://twitter.com/michelsforgov,michelsforgov,1514274154274512901
1,orgeon,https://twitter.com/ChristineDrazan,ChristineDrazan,965763444627550210
2,nevada,https://twitter.com/JoeLombardoNV,JoeLombardoNV,1389345377137176579
3,arizona,https://twitter.com/KariLake,KariLake,1082197856
4,kansas,https://twitter.com/DerekSchmidtKS,DerekSchmidtKS,161138443


## Get User Details

In [ ]:
# Specific which User details

my_user_fields = ['created_at','description','location','protected','profile_image_url','public_metrics','url','verified']

user_id

### Democrats

In [600]:
# Define DataFrame columns for all the User data elements we want to collect

govs_d_user_df = pd.DataFrame(columns=['screen_name',
                                       'id',
                                       'name',
                                       'created_at',
                                       'description',
                                       'location',
                                       'protected',
                                       'profile_image_url',
                                       'followers_count',
                                       'following_count',
                                       'tweet_count',
                                       'listed_count',
                                       'url',
                                       'verified'])

In [646]:
# Get User data for each Dem Governor

govs_d_id_ls = govs_d_df.id.to_list()

for i,agov_id in enumerate(govs_d_id_ls):
    print(f'Processing #{i}: {agov_id}')
    try:
        # https://docs.tweepy.org/en/v4.8.0/api.html#tweepy.API.get_user
        tweepy_user_obj = client.get_user(id=agov_id, user_fields=my_user_fields)
        print(f'    Twitter ID: {agov_id}')
        
        try:
            value_created_at = tweepy_user_obj.data.data['created_at']
        except:
            print('      NULL created_at')
            value_created_at = ''   
                 
        try:
            value_description = tweepy_user_obj.data.data['description']
        except:
            print('      NULL description')
            value_description = ''   
                 
        try:
            value_location = tweepy_user_obj.data.data['location']
        except:
            print('      NULL location')
            value_location = '' 
                   
        try:
            value_protected = tweepy_user_obj.data.data['protected']
        except:
            print('      NULL protected')
            value_protected = '' 
                   
        try:
            value_profile_image_url = tweepy_user_obj.data.data['profile_image_url']
        except:
            print('      NULL profile_image_url')
            value_profile_image_url = ''    
               
        try:
            value_url = tweepy_user_obj.data.data['url']
        except:
            print('      NULL url')
            value_url = ''   
                 
        try:
            value_followers_count = tweepy_user_obj.data.data['public_metrics']['followers_count']
        except:
            print('      NULL value_followers_count')
            value_followers_count = ''   
                 
        try:
            value_following_count = tweepy_user_obj.data.data['public_metrics']['following_count']
        except:
            print('      NULL value_following_count')
            value_following_count = ''   
                 
        try:
            value_tweet_count = tweepy_user_obj.data.data['public_metrics']['tweet_count']
        except:
            print('      NULL url')
            value_tweet_count = ''   
                 
        try:
            value_listed_count = tweepy_user_obj.data.data['public_metrics']['listed_count']
        except:
            print('      NULL listed_count')
            value_listed_count = ''   
                 
        try:
            value_verified = tweepy_user_obj.data.data['verified']
        except:
            print('      NULL verified')
            value_verified = ''        
        
        agov_user_info_ls = [tweepy_user_obj.data.data['username'],
                             tweepy_user_obj.data.data['id'],
                             tweepy_user_obj.data.data['name'],
                             value_created_at,
                             value_description,
                             value_location,
                             value_protected,
                             value_profile_image_url,
                             value_followers_count,
                             value_following_count,
                             value_tweet_count,
                             value_listed_count,
                             value_url,
                             value_verified
                             ] 
         
        print(f'    Appending row: {agov_user_info_ls}\n\n')
        govs_d_user_df.loc[len(govs_d_user_df)] = agov_user_info_ls                

        
    except:
        print(f'    ERROR: Could not retrieve Tweepy User Object')

Processing #0: 234896532
    Twitter ID: 234896532
    Appending row: ['GovEvers', '234896532', 'Governor Tony Evers', '2011-01-06T21:05:10.000Z', 'Husband to Kathy, lifelong educator, Euchre and bobblehead enthusiast, and 46th Governor of the State of Wisconsin.', 'Madison, Wisconsin', False, 'https://pbs.twimg.com/profile_images/1361681110955098113/4O5J0V8K_normal.jpg', 102554, 246, 7950, 899, 'https://t.co/JyH2JWfeyt', True]


Processing #1: 2835931010
    Twitter ID: 2835931010
    Appending row: ['TinaKotek', '2835931010', 'Tina Kotek', '2014-09-30T18:28:02.000Z', 'Progressive Democrat. Candidate for Oregon Governor. Committed to building a future of opportunity and justice for all Oregonians.', 'Oregon', False, 'https://pbs.twimg.com/profile_images/1500879380419002368/aPbCYJVn_normal.jpg', 19752, 339, 1629, 291, 'https://t.co/U0kpZIDyUZ', True]


Processing #2: 1080586433088258048
    Twitter ID: 1080586433088258048
    Appending row: ['GovSisolak', '1080586433088258048', 'Govern

In [647]:
# govs_d_user_df['id'] = govs_r_df['screen_name'].apply(lambda x: client.get_user(username=x).data.id)
govs_d_user_df.head()
govs_d_user_df.info()

,screen_name,id,name,created_at,description,location,protected,profile_image_url,followers_count,following_count,tweet_count,listed_count,url,verified
0,GovEvers,234896532,Governor Tony Evers,2011-01-06T21:05:10.000Z,"Husband to Kathy, lifelong educator, Euchre an...","Madison, Wisconsin",False,https://pbs.twimg.com/profile_images/136168111...,102548,246,7950,899,https://t.co/JyH2JWfeyt,True
1,katiehobbs,26659619,Katie Hobbs,2009-03-26T01:55:21.000Z,"Social worker, mom, wife, Arizona’s Secretary ...","Phoenix, AZ",False,https://pbs.twimg.com/profile_images/152449855...,134116,1576,10011,864,https://t.co/otdUvzR8kj,True
2,LauraKellyKS,364864453,Laura Kelly,2011-08-30T13:39:23.000Z,"48th Governor of Kansas, Wife, Mother, Champio...","Topeka, KS",False,https://pbs.twimg.com/profile_images/154147570...,54907,553,2778,390,https://t.co/ErZXA4bB3F,True
3,CentroNSR,957722624,Centro NSR,2012-11-19T16:03:53.000Z,"Twitter del Centro Ap. NSR, perteneciente al M...",Jr. los Pinos 291 - Camacho,False,https://pbs.twimg.com/profile_images/287018302...,324,42,1237,0,http://t.co/rEGFnaDF,False
4,staceyabrams,216065430,Stacey Abrams,2010-11-15T18:12:10.000Z,Democratic Nominee for Governor of Georgia. Sm...,Georgia,False,https://pbs.twimg.com/profile_images/150339247...,2915946,1928,15383,8786,https://t.co/KB1rUwIaan,True


<class 'pandas.core.frame.DataFrame'>
Int64Index: 111 entries, 0 to 110
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   screen_name        111 non-null    object
 1   id                 111 non-null    object
 2   name               111 non-null    object
 3   created_at         111 non-null    object
 4   description        111 non-null    object
 5   location           111 non-null    object
 6   protected          111 non-null    bool  
 7   profile_image_url  111 non-null    object
 8   followers_count    111 non-null    int64 
 9   following_count    111 non-null    int64 
 10  tweet_count        111 non-null    int64 
 11  listed_count       111 non-null    int64 
 12  url                111 non-null    object
 13  verified           111 non-null    bool  
dtypes: bool(2), int64(4), object(8)
memory usage: 11.5+ KB


In [649]:
# Save

filename_out = 'twitter_2022midterms_govs_d_user.txt'

govs_d_user_df.to_csv(filename_out)

### Republicans

In [650]:
govs_r_user_df = pd.DataFrame(columns=['screen_name',
                                       'id',
                                       'name',
                                       'created_at',
                                       'description',
                                       'location',
                                       'protected',
                                       'profile_image_url',
                                       'followers_count',
                                       'following_count',
                                       'tweet_count',
                                       'listed_count',
                                       'url',
                                       'verified'])

In [651]:
# Get User data for each Rep Governor

govs_r_id_ls = govs_r_df.id.to_list()

for i,agov_id in enumerate(govs_r_id_ls):
    print(f'Processing #{i}: {agov_id}')
    try:
        # https://docs.tweepy.org/en/v4.8.0/api.html#tweepy.API.get_user
        tweepy_user_obj = client.get_user(id=agov_id, user_fields=my_user_fields)
        print(f'    Twitter ID: {agov_id}')
        
        try:
            value_created_at = tweepy_user_obj.data.data['created_at']
        except:
            print('      NULL created_at')
            value_created_at = ''   
                 
        try:
            value_description = tweepy_user_obj.data.data['description']
        except:
            print('      NULL description')
            value_description = ''   
                 
        try:
            value_location = tweepy_user_obj.data.data['location']
        except:
            print('      NULL location')
            value_location = '' 
                   
        try:
            value_protected = tweepy_user_obj.data.data['protected']
        except:
            print('      NULL protected')
            value_protected = '' 
                   
        try:
            value_profile_image_url = tweepy_user_obj.data.data['profile_image_url']
        except:
            print('      NULL profile_image_url')
            value_profile_image_url = ''    
               
        try:
            value_url = tweepy_user_obj.data.data['url']
        except:
            print('      NULL url')
            value_url = ''   
                 
        try:
            value_followers_count = tweepy_user_obj.data.data['public_metrics']['followers_count']
        except:
            print('      NULL value_followers_count')
            value_followers_count = ''   
                 
        try:
            value_following_count = tweepy_user_obj.data.data['public_metrics']['following_count']
        except:
            print('      NULL value_following_count')
            value_following_count = ''   
                 
        try:
            value_tweet_count = tweepy_user_obj.data.data['public_metrics']['tweet_count']
        except:
            print('      NULL url')
            value_tweet_count = ''   
                 
        try:
            value_listed_count = tweepy_user_obj.data.data['public_metrics']['listed_count']
        except:
            print('      NULL listed_count')
            value_listed_count = ''   
                 
        try:
            value_verified = tweepy_user_obj.data.data['verified']
        except:
            print('      NULL verified')
            value_verified = ''        
        
        agov_user_info_ls = [tweepy_user_obj.data.data['username'],
                             tweepy_user_obj.data.data['id'],
                             tweepy_user_obj.data.data['name'],
                             value_created_at,
                             value_description,
                             value_location,
                             value_protected,
                             value_profile_image_url,
                             value_followers_count,
                             value_following_count,
                             value_tweet_count,
                             value_listed_count,
                             value_url,
                             value_verified
                             ] 
         
        print(f'    Appending row: {agov_user_info_ls}\n\n')
        govs_r_user_df.loc[len(govs_r_user_df)] = agov_user_info_ls                

        
    except:
        print(f'    ERROR: Could not retrieve Tweepy User Object')

Processing #0: 1514274154274512901
    Twitter ID: 1514274154274512901
    Appending row: ['michelsforgov', '1514274154274512901', 'Tim Michels', '2022-04-13T16:08:00.000Z', 'Army Veteran. Successful Businessman. Outsider.\n\nTim Michels is running for governor to get Wisconsin back on the right track.\n\nJoin the team!\nhttps://t.co/lHKqQnW72j', 'Wisconsin', False, 'https://pbs.twimg.com/profile_images/1537229738892304384/c_fOIcaW_normal.jpg', 11838, 165, 1009, 90, 'https://t.co/6hqLqdIXZh', True]


Processing #1: 965763444627550210
    Twitter ID: 965763444627550210
    Appending row: ['ChristineDrazan', '965763444627550210', 'Christine Drazan', '2018-02-20T01:42:02.000Z', 'Republican candidate for governor. I want to fix our schools, keep our communities safe, and make Oregon more affordable. Ready to lead #OnDay1.', 'Oregon, USA', False, 'https://pbs.twimg.com/profile_images/1569834457791856640/ndOE1hqb_normal.jpg', 18765, 345, 1124, 105, 'https://t.co/NXGQ3ZaQWN', True]


Processi

In [652]:
# govs_d_user_df['id'] = govs_r_df['screen_name'].apply(lambda x: client.get_user(username=x).data.id)
govs_r_user_df.head()
govs_r_user_df.info()

,screen_name,id,name,created_at,description,location,protected,profile_image_url,followers_count,following_count,tweet_count,listed_count,url,verified
0,michelsforgov,1514274154274512901,Tim Michels,2022-04-13T16:08:00.000Z,Army Veteran. Successful Businessman. Outsider...,Wisconsin,False,https://pbs.twimg.com/profile_images/153722973...,11838,165,1009,90,https://t.co/6hqLqdIXZh,True
1,ChristineDrazan,965763444627550210,Christine Drazan,2018-02-20T01:42:02.000Z,Republican candidate for governor. I want to f...,"Oregon, USA",False,https://pbs.twimg.com/profile_images/156983445...,18765,345,1124,105,https://t.co/NXGQ3ZaQWN,True
2,JoeLombardoNV,1389345377137176579,Joe Lombardo,2021-05-03T22:25:37.000Z,Army Veteran. Law Enforcement Officer of 34 ye...,"Nevada, USA",False,https://pbs.twimg.com/profile_images/140811208...,8113,35,957,74,https://t.co/SLKRLjxdld,True
3,KariLake,1082197856,Kari Lake,2013-01-12T07:47:51.000Z,Official Twitter Account for Trump-Endorsed Ca...,Iowa & Arizona,False,https://pbs.twimg.com/profile_images/156007313...,574295,981,24319,1118,https://t.co/EmzgORJKeQ,True
4,DerekSchmidtKS,161138443,Derek Schmidt,2010-06-30T02:39:27.000Z,Trusted Leader. Proven Winner. Kansas Conserva...,,False,https://pbs.twimg.com/profile_images/154952508...,4880,17,2130,62,https://t.co/ZYJ5mUrVYw,True


<class 'pandas.core.frame.DataFrame'>
Int64Index: 35 entries, 0 to 34
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   screen_name        35 non-null     object
 1   id                 35 non-null     object
 2   name               35 non-null     object
 3   created_at         35 non-null     object
 4   description        35 non-null     object
 5   location           35 non-null     object
 6   protected          35 non-null     bool  
 7   profile_image_url  35 non-null     object
 8   followers_count    35 non-null     int64 
 9   following_count    35 non-null     int64 
 10  tweet_count        35 non-null     int64 
 11  listed_count       35 non-null     int64 
 12  url                35 non-null     object
 13  verified           35 non-null     bool  
dtypes: bool(2), int64(4), object(8)
memory usage: 3.6+ KB


In [653]:
# Save

filename_out = 'twitter_2022midterms_govs_r_user.txt'

govs_r_user_df.to_csv(filename_out)

## Tests: Annotations/NER

In [684]:
type(client.get_users_followers(id=64614474))

tweepy.client.Response

In [691]:
query = "datascience"

new_tweets = client.search_recent_tweets(query=query, tweet_fields=['context_annotations', 'created_at'], max_results=10)
new_tweets

Response(data=[<Tweet id=1590272125977649152 text='RT @ratingology: 𝐒𝐭𝐚𝐭𝐬 𝐨𝐟 𝐥𝐢𝐯𝐞 𝐯𝐢𝐞𝐰𝐞𝐫𝐬 𝐨𝐧 #Youtube of #EnglishMedia #channels at 12:30pm Date :09/November/2022\n\n#Ratingology #Mediastats #…'>, <Tweet id=1590272074215751680 text='RT @ratingology: 𝐒𝐭𝐚𝐭𝐬 𝐨𝐟 𝐥𝐢𝐯𝐞 𝐯𝐢𝐞𝐰𝐞𝐫𝐬 𝐨𝐧 #Youtube of #HindiMedia #channels at #12PM Date : 09/November/2022\n\n#Ratingology #Mediastats #Rat…'>, <Tweet id=1590271964136230912 text='RT @mdancho84: One of the many reasons I chose #R. \n\nTrue story. 🧵\n\n#rstats #datascience https://t.co/lzSJ8Ce7QW'>, <Tweet id=1590271943080411136 text='RT @jaemciti: Become a Spatial #Data Scientist\n#ArtificialIntelligence #AI #ML #DataScience #DataScientists #CodeNewbies #Tech #deeplearnin…'>, <Tweet id=1590271926177406976 text='RT @waris027: https://t.co/HA5QE9xYAW\n\n#BigData #Analytics #DataScience #AI #MachineLearning #IoT #IIoT #Python #RStats #TensorFlow #JavaSc…'>, <Tweet id=1590271923027677184 text='RT @Amark2627: Download it&gt;https://t.co/EhewfF6p8t\n#serverless

In [693]:
for tweet in new_tweets.data:
    print("Text: ", tweet.text)
    print("Created at: ", tweet.created_at)
    print("\n")

Text:  RT @ratingology: 𝐒𝐭𝐚𝐭𝐬 𝐨𝐟 𝐥𝐢𝐯𝐞 𝐯𝐢𝐞𝐰𝐞𝐫𝐬 𝐨𝐧 #Youtube of #EnglishMedia #channels at 12:30pm Date :09/November/2022

#Ratingology #Mediastats #…
Created at:  2022-11-09 09:16:35+00:00


Text:  RT @ratingology: 𝐒𝐭𝐚𝐭𝐬 𝐨𝐟 𝐥𝐢𝐯𝐞 𝐯𝐢𝐞𝐰𝐞𝐫𝐬 𝐨𝐧 #Youtube of #HindiMedia #channels at #12PM Date : 09/November/2022

#Ratingology #Mediastats #Rat…
Created at:  2022-11-09 09:16:22+00:00


Text:  RT @mdancho84: One of the many reasons I chose #R. 

True story. 🧵

#rstats #datascience https://t.co/lzSJ8Ce7QW
Created at:  2022-11-09 09:15:56+00:00


Text:  RT @jaemciti: Become a Spatial #Data Scientist
#ArtificialIntelligence #AI #ML #DataScience #DataScientists #CodeNewbies #Tech #deeplearnin…
Created at:  2022-11-09 09:15:51+00:00


Text:  RT @waris027: https://t.co/HA5QE9xYAW

#BigData #Analytics #DataScience #AI #MachineLearning #IoT #IIoT #Python #RStats #TensorFlow #JavaSc…
Created at:  2022-11-09 09:15:47+00:00


Text:  RT @Amark2627: Download it&gt;https://t.co/EhewfF6p8t
#serverless 
#CodeNewb

In [694]:
for tweet in new_tweets.data:
    print(tweet.text)
    if len(tweet.context_annotations) > 0:
        print("\n")
        print(tweet.context_annotations)

RT @ratingology: 𝐒𝐭𝐚𝐭𝐬 𝐨𝐟 𝐥𝐢𝐯𝐞 𝐯𝐢𝐞𝐰𝐞𝐫𝐬 𝐨𝐧 #Youtube of #EnglishMedia #channels at 12:30pm Date :09/November/2022

#Ratingology #Mediastats #…


[{'domain': {'id': '131', 'name': 'Unified Twitter Taxonomy', 'description': 'A taxonomy of user interests. '}, 'entity': {'id': '1196446161223028736', 'name': 'Social media'}}, {'domain': {'id': '165', 'name': 'Technology', 'description': 'for individual and types of technology, e.g., food technology, 3D printing'}, 'entity': {'id': '1196446161223028736', 'name': 'Social media'}}, {'domain': {'id': '30', 'name': 'Entities [Entity Service]', 'description': 'Entity Service top level domain, every item that is in Entity Service should be in this domain'}, 'entity': {'id': '781974596752842752', 'name': 'Services'}}, {'domain': {'id': '47', 'name': 'Brand', 'description': 'Brands and Companies'}, 'entity': {'id': '10029382357', 'name': 'YouTube'}}, {'domain': {'id': '131', 'name': 'Unified Twitter Taxonomy', 'description': 'A taxonomy of user intere

In [696]:
import requests

In [697]:
client1 = tweepy.Client(bearer_token = bearer_token, return_type = requests.Response)
client1

In [698]:
new_tweets = client1.search_recent_tweets(query=query, tweet_fields=['context_annotations', 'created_at'], max_results=10)
new_tweets

<Response [200]>

In [699]:
tweets_json = new_tweets.json()
tweets_json

{'data': [{'edit_history_tweet_ids': ['1590272578983428097'],
   'context_annotations': [{'domain': {'id': '46',
      'name': 'Business Taxonomy',
      'description': 'Categories within Brand Verticals that narrow down the scope of Brands'},
     'entity': {'id': '1557697333571112960',
      'name': 'Technology Business',
      'description': 'Brands, companies, advertisers and every non-person handle with the profit intent related to softwares, apps, communication equipments, hardwares'}},
    {'domain': {'id': '30',
      'name': 'Entities [Entity Service]',
      'description': 'Entity Service top level domain, every item that is in Entity Service should be in this domain'},
     'entity': {'id': '848920371311001600',
      'name': 'Technology',
      'description': 'Technology and computing'}},
    {'domain': {'id': '66',
      'name': 'Interests and Hobbies Category',
      'description': 'A grouping of interests and hobbies entities, like Novelty Food or Destinations'},
     'e

In [700]:
tweets_data = tweets_json['data'] 
df = pd.json_normalize(tweets_data) 
df

,edit_history_tweet_ids,context_annotations,created_at,text,id
0,[1590272578983428097],"[{'domain': {'id': '46', 'name': 'Business Tax...",2022-11-09T09:18:23.000Z,RT @MarganJerry: Hire a Phone Expert for your ...,1590272578983428097
1,[1590272570443579395],"[{'domain': {'id': '46', 'name': 'Business Tax...",2022-11-09T09:18:21.000Z,RT @MarganJerry: Hire a Phone Expert for your ...,1590272570443579395
2,[1590272537937989635],"[{'domain': {'id': '10', 'name': 'Person', 'de...",2022-11-09T09:18:13.000Z,Hire a Phone Expert for your security\n#cybers...,1590272537937989635
3,[1590272490105815040],"[{'domain': {'id': '46', 'name': 'Business Tax...",2022-11-09T09:18:02.000Z,Interested in Data? Follow Me on Medium for We...,1590272490105815040
4,[1590272473974833152],"[{'domain': {'id': '69', 'name': 'News Vertica...",2022-11-09T09:17:58.000Z,RT @quepasachico: Start a #Digital Detox. I ne...,1590272473974833152
5,[1590272460725026818],"[{'domain': {'id': '46', 'name': 'Business Tax...",2022-11-09T09:17:55.000Z,RT @Amark2627: Download it &gt;&gt;https://t.c...,1590272460725026818
6,[1590272427141234689],"[{'domain': {'id': '46', 'name': 'Business Tax...",2022-11-09T09:17:47.000Z,RT @Amark2627: Download it &gt;&gt;https://t.c...,1590272427141234689
7,[1590272425694224384],"[{'domain': {'id': '46', 'name': 'Business Tax...",2022-11-09T09:17:46.000Z,RT @Amark2627: Download it &gt;&gt;https://t.c...,1590272425694224384
8,[1590272403888037888],"[{'domain': {'id': '46', 'name': 'Business Tax...",2022-11-09T09:17:41.000Z,Download it &gt;&gt;https://t.co/EhewfF6p8t\n#...,1590272403888037888
9,[1590272396920954880],"[{'domain': {'id': '46', 'name': 'Business Tax...",2022-11-09T09:17:39.000Z,RT @Amark2627: Download it &gt;&gt;https://t.c...,1590272396920954880


In [688]:
response = client.search_recent_tweets(
    "innovation -is:retweet lang:pl",
    max_results = 10,
    tweet_fields = ['author_id','created_at','text','source','lang','geo'],
    user_fields = ['name','username','location','verified'],
    expansions = ['geo.place_id', 'author_id'],
    place_fields = ['country','country_code']
)


for tweet in response.data:
    print(tweet.author_id)      # print the author id of the tweet
    print(tweet.text)           # print the text
    print(tweet.data['lang'])   # print the language (PL, since we're filtering by it)
    print(tweet.data['source']) # what did the user use to publish the tweet?
    


1501850873529573377
Rejestracja na Brussels Talks: "Europejski Instytut Innowacji i Technologii – kluczowe wyzwania przed nami" 
🗓️17 Listopada 2022 r.: 
⏰14:00-15:00

link ➡️https://t.co/tXKguEzgnx

Zachęcamy do uczestnictwa (2/2)
#HoryzontEU #EIT #innowacje #technologie #badania #biznes #nauka
pl
Twitter Web App
1938490717
Finowie wykorzystują model AIG (Anticipatory Innovation Governance) autorstwa @OPSIgov  do przygotowania się do wyzwań, które mogą pojawić się w jeszcze nieznanej przyszłości. Jednym z elementów takiego podejścia jest program dbania o well-being dzieci. 
https://t.co/qS95LmabsL https://t.co/OAcji4nBlc
pl
Twitter Web App
1095246745628561408
📍Promujemy reklamę dla małych, jak i dużych przedsiębiorstw. Ułatwiamy reklamodawcom dotarcie do odpowiednich konsumentów.

#GutenburgMedia

📌Można nas spotkać wszędzie.

#advertising #marketing #morden #polska #warszawa #innovation #dooh #reklama #promotion #biznes #twitter https://t.co/TE6BbICRZP
pl
Twitter for iPhone
42627979


In [689]:
tweets_json = response.json()
tweets_json

tweets_data = tweets_json['data'] 
df = pd.json_normalize(tweets_data) 
df.head()

AttributeError: 'Response' object has no attribute 'json'

In [679]:
test_obj = client.get_users_tweets(id=64614474, exclude=['retweets'], expansions=['author_id'], 
                                   max_results=5, tweet_fields=['created_at','geo','public_metrics'])
test_obj.data
test_obj.includes

[<Tweet id=1574887372281348098 text='https://t.co/v6KHSnaSNC https://t.co/KYfnWj4MQe'>,
 <Tweet id=1569061551444996097 text='May God’s Blessings and Grace continue to shine on all us. https://t.co/wZUhLdtExv'>,
 <Tweet id=1568131080989048832 text='https://t.co/qn1YFYMVrp'>,
 <Tweet id=1566288759506681856 text='Had a great time seeing old friends, familiar faces, meeting new people, and learning more about the Okinawan Culture at the Okinawan Festival today! Mahalo Dave, Lavern, Yuri, and everyone that came up to us saying we have your vote… mahalo! 😄🤙🏼 #AionaTupai2022 https://t.co/MZEmo85yv3'>,
 <Tweet id=1548914554427432960 text='Kaneohe meet &amp; greet with friends and supporters.  #dukeaiona2022 #hawaii https://t.co/ejkEu8pWU6'>]

{'users': [<User id=64614474 name=Duke Aiona username=DukeAiona2022>]}

## Get User Timelines

In [702]:
govs_d_user_df.head(2)
govs_d_user_df.info()

govs_d_id_ls = govs_d_user_df.id.to_list()
govs_d_id_ls

,screen_name,id,name,created_at,description,location,protected,profile_image_url,followers_count,following_count,tweet_count,listed_count,url,verified
0,GovEvers,234896532,Governor Tony Evers,2011-01-06T21:05:10.000Z,"Husband to Kathy, lifelong educator, Euchre an...","Madison, Wisconsin",False,https://pbs.twimg.com/profile_images/136168111...,102548,246,7950,899,https://t.co/JyH2JWfeyt,True
1,katiehobbs,26659619,Katie Hobbs,2009-03-26T01:55:21.000Z,"Social worker, mom, wife, Arizona’s Secretary ...","Phoenix, AZ",False,https://pbs.twimg.com/profile_images/152449855...,134116,1576,10011,864,https://t.co/otdUvzR8kj,True


<class 'pandas.core.frame.DataFrame'>
Int64Index: 111 entries, 0 to 110
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   screen_name        111 non-null    object
 1   id                 111 non-null    object
 2   name               111 non-null    object
 3   created_at         111 non-null    object
 4   description        111 non-null    object
 5   location           111 non-null    object
 6   protected          111 non-null    bool  
 7   profile_image_url  111 non-null    object
 8   followers_count    111 non-null    int64 
 9   following_count    111 non-null    int64 
 10  tweet_count        111 non-null    int64 
 11  listed_count       111 non-null    int64 
 12  url                111 non-null    object
 13  verified           111 non-null    bool  
dtypes: bool(2), int64(4), object(8)
memory usage: 11.5+ KB


['234896532',
 '26659619',
 '364864453',
 '957722624',
 '216065430',
 '24209167',
 '38970940',
 '342863309',
 '158238919',
 '600422073',
 '45550693',
 '106773922',
 '36231074',
 '40296465',
 '11347122',
 '110573669',
 '64614474',
 '17622662',
 '545107258',
 '234896532',
 '26659619',
 '364864453',
 '957722624',
 '216065430',
 '24209167',
 '38970940',
 '342863309',
 '158238919',
 '600422073',
 '45550693',
 '106773922',
 '36231074',
 '40296465',
 '11347122',
 '110573669',
 '64614474',
 '17622662',
 '545107258',
 '234896532',
 '26659619',
 '364864453',
 '957722624',
 '216065430',
 '24209167',
 '38970940',
 '342863309',
 '158238919',
 '600422073',
 '45550693',
 '106773922',
 '36231074',
 '40296465',
 '11347122',
 '110573669',
 '64614474',
 '17622662',
 '545107258',
 '234896532',
 '26659619',
 '364864453',
 '957722624',
 '216065430',
 '24209167',
 '38970940',
 '342863309',
 '158238919',
 '600422073',
 '45550693',
 '106773922',
 '36231074',
 '40296465',
 '11347122',
 '110573669',
 '64614474',

In [705]:
len(govs_d_user_df)

111

In [709]:
govs_d_user_df.drop_duplicates(inplace=True)
govs_d_user_df.shape

govs_d_id_ls = govs_d_user_df.id.to_list()

(68, 14)

In [712]:
# Democrats 
tweets_ls = []

for i, user_id in enumerate(govs_d_id_ls):
    print(f'User #{i}, ID: {user_id}')
    tweets_obj = client.get_users_tweets(user_id, tweet_fields=['created_at','public_metrics','context_annotations','entities'], max_results=10)
    tweets_ls.append(tweets_obj)

User #0, ID: 234896532
User #1, ID: 26659619
User #2, ID: 364864453
User #3, ID: 957722624
User #4, ID: 216065430
User #5, ID: 24209167
User #6, ID: 38970940
User #7, ID: 342863309
User #8, ID: 158238919
User #9, ID: 600422073
User #10, ID: 45550693
User #11, ID: 106773922
User #12, ID: 36231074
User #13, ID: 40296465
User #14, ID: 11347122
User #15, ID: 110573669
User #16, ID: 64614474
User #17, ID: 17622662
User #18, ID: 545107258
User #19, ID: 26659619
User #20, ID: 216065430
User #21, ID: 24209167
User #22, ID: 38970940
User #23, ID: 342863309
User #24, ID: 11347122
User #25, ID: 234896532
User #26, ID: 26659619
User #27, ID: 216065430
User #28, ID: 24209167
User #29, ID: 38970940
User #30, ID: 342863309
User #31, ID: 600422073
User #32, ID: 45550693
User #33, ID: 36231074
User #34, ID: 11347122
User #35, ID: 234896532
User #36, ID: 216065430
User #37, ID: 24209167
User #38, ID: 342863309
User #39, ID: 600422073
User #40, ID: 40296465
User #41, ID: 11347122
User #42, ID: 234896532


In [732]:
pprint(tweets_ls[1].data)

[<Tweet id=1590187119515152384 text="My statement on awaiting tonight's results. https://t.co/yba7ASq3i6">,
 <Tweet id=1590170247369089024 text='Kari Lake has spent the last two years spreading lies about our safe and secure elections. So here are the facts about how elections actually work in our great state.\n\nWATCH THIS &gt;&gt; https://t.co/5DaOc9XBSG'>,
 <Tweet id=1590159739291303936 text='If you’re in line to vote by 7pm, stay in line!\n\nYou have a right to cast your ballot.'>,
 <Tweet id=1590147156622512128 text='POLLS CLOSE IN ONE HOUR!\n\nIf you haven’t voted already, now is your final chance to protect our democracy and make your voice heard.'>,
 <Tweet id=1590128282296467456 text='This election is a choice between sanity and chaos. \n\nKari Lake has no real plans to make Arizona work better for families — she only has dangerous rhetoric and conspiracy theories. \n\nThere are only three hours left to defeat her.'>,
 <Tweet id=1590119860725743616 text='RT @rweingarten: Arizo

In [657]:
# CAUTION: Long-running process, normal OAuth can only retrieve 900/15min rate

def get_user_tweets(username):
    # global api, tweets_govs_df,count,error
    
    try:
        # API v1.0
        tweets = api.search_all_tweets(screen_name=username, 
                            include_rts = False,
                            tweet_mode = 'extended',
                            )
        table = [[tweet._json['full_text'],username, political_party, tweet._json['in_reply_to_user_id'],tweet._json['entities']['user_mentions'],tweet._json['retweet_count'], tweet._json['favorite_count'],tweet._json['created_at']] for  tweet in tweets]
        df = pd.DataFrame(table)
        df.columns = ['text', 'author', 'political_party','reply_to', 'user_mentions','retweet_count', 'favorite_count', 'created_at']
        tweets_govs_df = tweets_govs_df.append(df)
    except:
        error+=1
        print(error, count)
        if count%100==True:
            print(count)

# Option 3: URL Endpoints: Twitter v2.0 API

Regular 

* https://github.com/twitterdev/Twitter-API-v2-sample-code

Academic

* https://github.com/twitterdev/getting-started-with-the-twitter-api-v2-for-academic-research/blob/main/modules/3-deciding-which-endpoints-to-use.md

In [55]:
%whos DataFrame

Variable          Type         Data/Info
----------------------------------------
govs_d_df         DataFrame                 state       <...>dtforgov      heidtforgov
govs_d_user_df    DataFrame    Empty DataFrame\nColumns:<...> listed_count]\nIndex: []
govs_df           DataFrame                 state       <...>twitter.com/heidtforgov  
govs_r_df         DataFrame                 state       <...>alABundy       RealABundy
govs_r_user_df    DataFrame    Empty DataFrame\nColumns:<...> listed_count]\nIndex: []
my_followers_df   DataFrame                             <...>n\n[100 rows x 4 columns]
my_following_df   DataFrame                id           <...>n\n[100 rows x 3 columns]
my_likes_df       DataFrame                             <...>\n\n[97 rows x 5 columns]
my_timeline_df    DataFrame                             <...>022-10-27T16:00:01.000Z  


In [56]:
# ONLY RUN if you skipped API v1/v2 search and merge above

govs_d_user_df = govs_d_df.copy()
govs_r_user_df = govs_r_df.copy()

In [57]:
govs_d_user_df.head()
govs_d_user_df.info()

,state,democrat,screen_name
0,wisconsin,https://twitter.com/GovEvers,GovEvers
1,orgeon,https://twitter.com/TinaKotek,TinaKotek
2,nevada,https://twitter.com/GovSisolak,GovSisolak
3,arizona,https://twitter.com/katiehobbs,katiehobbs
4,kansas,https://twitter.com/LauraKellyKS,LauraKellyKS


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   state        35 non-null     object
 1   democrat     35 non-null     object
 2   screen_name  35 non-null     object
dtypes: object(3)
memory usage: 968.0+ bytes


In [58]:
def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2UserLookupPython"
    return r

In [59]:
def connect_to_endpoint_wo_params(url):
    response = requests.request("GET", url, auth=bearer_oauth,)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()

In [60]:
def connect_to_endpoint_with_params(url, tweet_fields):
    response = requests.request(
        "GET", url, auth=bearer_oauth, params=tweet_fields)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()

### User Info

In [61]:
# Define DataFrame columns for all the User data elements we want to collect

govs_d_user_df = pd.DataFrame(columns=['screen_name',
                                    'id',
                                    'name',
                                    'location',
                                    'description',
                                    'url',
                                    'created_at',
                                    'verified',
                                    'protected',
                                    # 'followers',
                                    'followers_count',
                                    # 'followers_id',
                                    # 'following',
                                    'follow_request_sent',
                                    # 'follow',
                                    # 'friends',
                                    'friends_count',
                                    'favorites_count',
                                    'listed_count',
                                    # 'status',
                                    # 'statuses_count',
                                    ])

govs_r_user_df = pd.DataFrame(columns=['screen_name',
                                    'id',
                                    'name',
                                    'location',
                                    'description',
                                    'url',
                                    'created_at',
                                    'verified',
                                    'protected',
                                    # 'followers',
                                    'followers_count',
                                    # 'followers_id',
                                    # 'following',
                                    'follow_request_sent',
                                    # 'follow',
                                    # 'friends',
                                    'friends_count',
                                    'favorites_count',
                                    'listed_count',
                                    # 'status',
                                    # 'statuses_count',
                                    ])

In [62]:
govs_d_df.head()

,state,democrat,screen_name
0,wisconsin,https://twitter.com/GovEvers,GovEvers
1,orgeon,https://twitter.com/TinaKotek,TinaKotek
2,nevada,https://twitter.com/GovSisolak,GovSisolak
3,arizona,https://twitter.com/katiehobbs,katiehobbs
4,kansas,https://twitter.com/LauraKellyKS,LauraKellyKS


In [45]:
def connect_to_endpoint_with_params(url, tweet_fields):
    response = requests.request(
        "GET", url, auth=bearer_oauth, params=tweet_fields)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()

In [65]:
def get_user_by_username(username, param_str):
    '''
    
    '''
    url = f'https://api.twitter.com/2/users/by/username/{username}?user.fields={param_str}'
    url = f"https://api.twitter.com/2/users/by?user.fields={params}"
    params = {"user.fields": param_str}
    json_response = connect_to_endpoint_with_params(url, params)
    return json_response
    
# Test

username = 'KariLake'
params = 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld'

user_id = 234896532
json_response = get_followers(user_id)
print(json.dumps(json_response, indent=4, sort_keys=True))

my_followers_df = pd.DataFrame(json_response['data'])
my_followers_df.head()
my_followers_df.info()



url = 'https://api.twitter.com/2/users/by?user.fields='
url


200
{
    "data": [
        {
            "created_at": "2021-05-04T19:31:07.000Z",
            "id": "1389663738945495040",
            "name": "Dr. Jill Underly, WI State Superintendent",
            "username": "DrJillUnderly"
        },
        {
            "created_at": "2021-01-06T17:02:07.000Z",
            "id": "1346864195304792065",
            "name": "Senator Brad Pfaff",
            "username": "SenBradPfaff"
        },
        {
            "created_at": "2009-05-27T19:35:11.000Z",
            "id": "42953875",
            "name": "Western Tech College",
            "username": "WesternTC"
        },
        {
            "created_at": "2008-07-21T18:48:42.000Z",
            "id": "15518969",
            "name": "Ducks Unlimited",
            "username": "DucksUnlimited"
        },
        {
            "created_at": "2018-05-01T14:15:25.000Z",
            "id": "991320189634478080",
            "name": "Forward Madison FC",
            "username": "ForwardMSNFC"
       

,name,username,id,created_at
0,"Dr. Jill Underly, WI State Superintendent",DrJillUnderly,1389663738945495040,2021-05-04T19:31:07.000Z
1,Senator Brad Pfaff,SenBradPfaff,1346864195304792065,2021-01-06T17:02:07.000Z
2,Western Tech College,WesternTC,42953875,2009-05-27T19:35:11.000Z
3,Ducks Unlimited,DucksUnlimited,15518969,2008-07-21T18:48:42.000Z
4,Forward Madison FC,ForwardMSNFC,991320189634478080,2018-05-01T14:15:25.000Z


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        100 non-null    object
 1   username    100 non-null    object
 2   id          100 non-null    object
 3   created_at  100 non-null    object
dtypes: object(4)
memory usage: 3.2+ KB


'https://api.twitter.com/2/users/by?user.fields='

In [43]:
# Get User data for each Dem Governor

for i,agov_username in enumerate(govs_d_df.screen_name.to_list()):
    print(f'Processing #{i}: {agov_username}')
    try:
        # https://docs.tweepy.org/en/v4.8.0/api.html#tweepy.API.get_user
        tweepy_user_obj = api.get_user(screen_name=agov_username)
        print(f'    Twitter ID: {agov_id}')
        
        agov_user_info_ls = [tweepy_user_obj.screen_name,
                             tweepy_user_obj.id,
                             tweepy_user_obj.name,
                             tweepy_user_obj.location,
                             tweepy_user_obj.description,
                             tweepy_user_obj.url,
                             tweepy_user_obj.created_at,
                             tweepy_user_obj.verified,
                             tweepy_user_obj.protected,
                             # tweepy_user_obj.followers,
                             tweepy_user_obj.followers_count,
                             # tweepy_user_obj.followers_ids,
                             # tweepy_user_obj.following,
                             tweepy_user_obj.follow_request_sent,
                             # tweepy_user_obj.follow,
                             # tweepy_user_obj.friends,
                             tweepy_user_obj.friends_count,
                             tweepy_user_obj.favourites_count,
                             tweepy_user_obj.listed_count,
                             # tweepy_user_obj.status,
                             # tweepy_user_obj.statuses_count
                             ]  
        print(f'    Appending row: {agov_user_info_ls}\n\n')
        govs_d_user_df.loc[len(govs_d_user_df)] = agov_user_info_ls                

        
    except:
        print(f'    ERROR: Could not retrieve Tweepy User Object')


Processing #0: GovEvers
    ERROR: Could not retrieve Tweepy User Object
Processing #1: TinaKotek
    ERROR: Could not retrieve Tweepy User Object
Processing #2: GovSisolak
    ERROR: Could not retrieve Tweepy User Object
Processing #3: katiehobbs
    ERROR: Could not retrieve Tweepy User Object
Processing #4: LauraKellyKS
    ERROR: Could not retrieve Tweepy User Object
Processing #5: GovMLG
    ERROR: Could not retrieve Tweepy User Object
Processing #6: gretchenwhitmer
    ERROR: Could not retrieve Tweepy User Object
Processing #7: GovJanetMills
    ERROR: Could not retrieve Tweepy User Object
Processing #8: staceyabrams
    ERROR: Could not retrieve Tweepy User Object
Processing #9: GovTimWalz
    ERROR: Could not retrieve Tweepy User Object
Processing #10: JoshShapiroPA
    ERROR: Could not retrieve Tweepy User Object
Processing #11: joy4ok
    ERROR: Could not retrieve Tweepy User Object
Processing #12: CharlieCrist
    ERROR: Could not retrieve Tweepy User Object
Processing #13: 

In [25]:
# 

# Specify the usernames that you want to lookup below
# You can enter up to 100 comma-separated values.
usernames = "usernames=TwitterDev,TwitterAPI"
user_fields = "user.fields=description,created_at"
# User fields are adjustable, options include:
# created_at, description, entities, id, location, name,
# pinned_tweet_id, profile_image_url, protected,
# public_metrics, url, username, verified, and withheld
url = "https://api.twitter.com/2/users/by?{}&{}".format(usernames, user_fields)
    
json_response = connect_to_endpoint_wo_params(url)
print(json.dumps(json_response, indent=4, sort_keys=True))

200
{
    "data": [
        {
            "created_at": "2013-12-14T04:35:55.000Z",
            "description": "The voice of the #TwitterDev team and your official source for updates, news, and events, related to the #TwitterAPI.",
            "id": "2244994945",
            "name": "Twitter Dev",
            "username": "TwitterDev"
        },
        {
            "created_at": "2007-05-23T06:01:13.000Z",
            "description": "Tweets about changes and service issues. Follow @TwitterDev\u00a0for more.",
            "id": "6253282",
            "name": "Twitter API",
            "username": "TwitterAPI"
        }
    ]
}


### User Followers

In [66]:
def get_followers(user_id):
    url = "https://api.twitter.com/2/users/{}/following".format(user_id)
    params = {"user.fields": "created_at"}
    json_response = connect_to_endpoint_with_params(url, params)
    return json_response
    
# Test
user_id = 234896532
json_response = get_followers(user_id)
print(json.dumps(json_response, indent=4, sort_keys=True))

my_followers_df = pd.DataFrame(json_response['data'])
my_followers_df.head()
my_followers_df.info()
    

200
{
    "data": [
        {
            "created_at": "2021-05-04T19:31:07.000Z",
            "id": "1389663738945495040",
            "name": "Dr. Jill Underly, WI State Superintendent",
            "username": "DrJillUnderly"
        },
        {
            "created_at": "2021-01-06T17:02:07.000Z",
            "id": "1346864195304792065",
            "name": "Senator Brad Pfaff",
            "username": "SenBradPfaff"
        },
        {
            "created_at": "2009-05-27T19:35:11.000Z",
            "id": "42953875",
            "name": "Western Tech College",
            "username": "WesternTC"
        },
        {
            "created_at": "2008-07-21T18:48:42.000Z",
            "id": "15518969",
            "name": "Ducks Unlimited",
            "username": "DucksUnlimited"
        },
        {
            "created_at": "2018-05-01T14:15:25.000Z",
            "id": "991320189634478080",
            "name": "Forward Madison FC",
            "username": "ForwardMSNFC"
       

,id,username,name,created_at
0,1389663738945495040,DrJillUnderly,"Dr. Jill Underly, WI State Superintendent",2021-05-04T19:31:07.000Z
1,1346864195304792065,SenBradPfaff,Senator Brad Pfaff,2021-01-06T17:02:07.000Z
2,42953875,WesternTC,Western Tech College,2009-05-27T19:35:11.000Z
3,15518969,DucksUnlimited,Ducks Unlimited,2008-07-21T18:48:42.000Z
4,991320189634478080,ForwardMSNFC,Forward Madison FC,2018-05-01T14:15:25.000Z


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          100 non-null    object
 1   username    100 non-null    object
 2   name        100 non-null    object
 3   created_at  100 non-null    object
dtypes: object(4)
memory usage: 3.2+ KB


### User Following

In [67]:
def get_user_following(id):
    url = "https://api.twitter.com/2/users/{}/following".format(user_id)
    json_response = connect_to_endpoint_wo_params(url)
    return json_response

# Test
user_id = 2244994945

json_response = get_user_following(user_id)
print(json.dumps(json_response, indent=4, sort_keys=True))

my_following_df = pd.DataFrame(json_response['data'])
my_following_df.head()
my_following_df.info()

200
{
    "data": [
        {
            "id": "44196397",
            "name": "Elon Musk",
            "username": "elonmusk"
        },
        {
            "id": "633050374",
            "name": "Aba Micah",
            "username": "2006legend"
        },
        {
            "id": "15772978",
            "name": "Jessica Garson",
            "username": "jessicagarson"
        },
        {
            "id": "4839528448",
            "name": "Bitcoin Core Project",
            "username": "bitcoincoreorg"
        },
        {
            "id": "25029451",
            "name": "blue checkmark (real)",
            "username": "Punkaboo"
        },
        {
            "id": "1009484430250053632",
            "name": "cora",
            "username": "coraF256"
        },
        {
            "id": "108209516",
            "name": "Amir Shevat",
            "username": "ashevat"
        },
        {
            "id": "1062359582",
            "name": "Sheryl Klein Lavi\ud83c\udff4\ud

,id,name,username
0,44196397,Elon Musk,elonmusk
1,633050374,Aba Micah,2006legend
2,15772978,Jessica Garson,jessicagarson
3,4839528448,Bitcoin Core Project,bitcoincoreorg
4,25029451,blue checkmark (real),Punkaboo


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        100 non-null    object
 1   name      100 non-null    object
 2   username  100 non-null    object
dtypes: object(3)
memory usage: 2.5+ KB


### User Timeline

In [69]:
def get_user_timeline(id, params):
    url = "https://api.twitter.com/2/users/{}/tweets".format(id)
    # params = {"tweet.fields": "created_at"}
    json_response = connect_to_endpoint_with_params(url, params)
    return json_response


# Tweet fields are adjustable.
# Options include:
# attachments, author_id, context_annotations,
# conversation_id, created_at, entities, geo, id,
# in_reply_to_user_id, lang, non_public_metrics, organic_metrics,
# possibly_sensitive, promoted_metrics, public_metrics, referenced_tweets,
# source, text, and withheld
my_params = {"tweet.fields": "created_at"}

# Test
user_id = 2244994945

json_response = get_user_timeline(user_id, my_params)
print(json.dumps(json_response, indent=4, sort_keys=True))

my_timeline_df = pd.DataFrame(json_response['data'])
my_timeline_df.head()
my_timeline_df.info()

200
{
    "data": [
        {
            "created_at": "2022-11-02T23:15:29.000Z",
            "edit_history_tweet_ids": [
                "1587946527955329024"
            ],
            "id": "1587946527955329024",
            "text": "As always, we\u2019re just a Tweet away, so feel free to reach out with any questions. We\u2019re grateful for your partnership to #BuildWhatsNext"
        },
        {
            "created_at": "2022-11-02T23:15:29.000Z",
            "edit_history_tweet_ids": [
                "1587946526617264128"
            ],
            "id": "1587946526617264128",
            "text": "We\u2019ll still celebrate the soon-to-be-announced winners of our Chirp Developer Challenge - stay tuned for more details!"
        },
        {
            "created_at": "2022-11-02T23:15:28.000Z",
            "edit_history_tweet_ids": [
                "1587946525245816832"
            ],
            "id": "1587946525245816832",
            "text": "We\u2019re currently hard at

,text,edit_history_tweet_ids,created_at,id
0,"As always, we’re just a Tweet away, so feel fr...",[1587946527955329024],2022-11-02T23:15:29.000Z,1587946527955329024
1,We’ll still celebrate the soon-to-be-announced...,[1587946526617264128],2022-11-02T23:15:29.000Z,1587946526617264128
2,We’re currently hard at work to make Twitter b...,[1587946525245816832],2022-11-02T23:15:28.000Z,1587946525245816832
3,💡 #TipTuesday: Ever wondered how to get the v...,[1587519847281397767],2022-11-01T19:00:00.000Z,1587519847281397767
4,✍️Fill in the blank ⬇️\n\nI start my morning o...,[1587066866824085505],2022-10-31T13:00:01.000Z,1587066866824085505


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   text                    10 non-null     object
 1   edit_history_tweet_ids  10 non-null     object
 2   created_at              10 non-null     object
 3   id                      10 non-null     object
dtypes: object(4)
memory usage: 448.0+ bytes


### Liked Tweets

In [80]:
def get_liked_tweets(id, params):
    url = "https://api.twitter.com/2/users/{}/liked_tweets".format(id)
    # params = {"tweet.fields": "created_at"}
    json_response = connect_to_endpoint_with_params(url, params)
    return json_response


# Tweet fields are adjustable.
# Options include:
# attachments, author_id, context_annotations,
# conversation_id, created_at, entities, geo, id,
# in_reply_to_user_id, lang, non_public_metrics, organic_metrics,
# possibly_sensitive, promoted_metrics, public_metrics, referenced_tweets,
# source, text, and withheld
tweet_fields = "tweet.fields=lang,author_id"
# Be sure to replace your-user-id with your own user ID or one of an authenticating user
# You can find a user ID by using the user lookup endpoint
id = "your-user-id"
# You can adjust ids to include a single Tweets.
# Or you can add to up to 100 comma-separated IDs
url = "https://api.twitter.com/2/users/{}/liked_tweets".format(id)

# Test
user_id = 2244994945

json_response = get_liked_tweets(user_id, tweet_fields)
print(json.dumps(json_response, indent=4, sort_keys=True))

my_liked_tweets_df = pd.DataFrame(json_response['data'])
my_liked_tweets_df.head()
my_liked_tweets_df.info() 

200
{
    "data": [
        {
            "author_id": "1136175005060878337",
            "edit_history_tweet_ids": [
                "1581308268122877953"
            ],
            "id": "1581308268122877953",
            "lang": "en",
            "text": "@TwitterDev Tips and tricks would be cool!!"
        },
        {
            "author_id": "109628138",
            "edit_history_tweet_ids": [
                "1575158698225258500"
            ],
            "id": "1575158698225258500",
            "lang": "en",
            "text": "On a call with @TwitterDev team looking at the upcoming platform functionality \u2014 some exiting functionality in the works. \n\nAlso, I \u2764\ufe0f the approach twitter is taking to feature development. Good job, bird ppl"
        },
        {
            "author_id": "1120050519182016513",
            "edit_history_tweet_ids": [
                "1575117901647986691"
            ],
            "id": "1575117901647986691",
            "lang": "en",


,text,id,lang,edit_history_tweet_ids,author_id
0,@TwitterDev Tips and tricks would be cool!!,1581308268122877953,en,[1581308268122877953],1136175005060878337
1,On a call with @TwitterDev team looking at the...,1575158698225258500,en,[1575158698225258500],109628138
2,Big thanks @TwitterDev for these wonderful hig...,1575117901647986691,en,[1575117901647986691],1120050519182016513
3,Congratulations to @triketora and the @blockpa...,1572733815965253632,en,[1572733815965253632],108209516
4,"🎉 Thank you so much for the kind gesture, @Twi...",1571949434010992640,en,[1571949434010992640],1264433760


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   text                    97 non-null     object
 1   id                      97 non-null     object
 2   lang                    97 non-null     object
 3   edit_history_tweet_ids  97 non-null     object
 4   author_id               97 non-null     object
dtypes: object(5)
memory usage: 3.9+ KB


### Liking Users

In [74]:
json_response = get_liking_users(user_id, user_fields)
pprint(json_response)

200
{'meta': {'next_token': '7140dibdnow9c7btw420jlmkll17j0clkx1bl4w5gi6nr',
          'result_count': 0}}


In [81]:
def get_liking_users(id, params):
    url = "https://api.twitter.com/2/tweets/{}/liking_users".format(id)
    # params = {"tweet.fields": "created_at"}
    json_response = connect_to_endpoint_with_params(url, params)
    return json_response


# User fields are adjustable, options include:
# created_at, description, entities, id, location, name,
# pinned_tweet_id, profile_image_url, protected,
# public_metrics, url, username, verified, and withheld
user_fields = "user.fields=created_at,description"
# You can replace the ID given with the Tweet ID you wish to like.
# You can find an ID by using the Tweet lookup endpoint
id = "1590693373056122880"
# You can adjust ids to include a single Tweets.
# Or you can add to up to 100 comma-separated IDs

# Test
user_id = "1590693373056122880"

json_response = get_liking_users(user_id, user_fields)
print(json.dumps(json_response, indent=4, sort_keys=True))

my_liking_users_df = pd.DataFrame(json_response['data'])
my_liking_users_df.head()
my_liking_users_df.info() 

200
{
    "data": [
        {
            "created_at": "2011-10-15T22:25:54.000Z",
            "description": "",
            "id": "391659128",
            "name": "Dimitri Schmid",
            "username": "dimitrischmid"
        },
        {
            "created_at": "2015-03-31T15:54:56.000Z",
            "description": "Programming Officer at ONGC!!!",
            "id": "3120779995",
            "name": "ABHISHEK BHARDWAJ",
            "username": "abhishek21795"
        },
        {
            "created_at": "2020-04-29T19:27:35.000Z",
            "description": "",
            "id": "1255579413934030848",
            "name": "devesh_deepak",
            "username": "deveshdeepak9"
        },
        {
            "created_at": "2018-11-29T20:52:12.000Z",
            "description": "I enable insights using data and math. Builder of the contact index. AI consultant for German federal government. Views and titles (Dr. habil.) my own.",
            "id": "1068246270752538630",
     

,id,name,description,username,created_at
0,391659128,Dimitri Schmid,,dimitrischmid,2011-10-15T22:25:54.000Z
1,3120779995,ABHISHEK BHARDWAJ,Programming Officer at ONGC!!!,abhishek21795,2015-03-31T15:54:56.000Z
2,1255579413934030848,devesh_deepak,,deveshdeepak9,2020-04-29T19:27:35.000Z
3,1068246270752538630,Sten Rüdiger,I enable insights using data and math. Builder...,StenRuediger,2018-11-29T20:52:12.000Z
4,3319164733,Ifty Mohammad Rezwan,"Data Monger,\nAll opinions are my own.",imr165,2015-08-18T20:34:13.000Z


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           93 non-null     object
 1   name         93 non-null     object
 2   description  93 non-null     object
 3   username     93 non-null     object
 4   created_at   93 non-null     object
dtypes: object(5)
memory usage: 3.8+ KB


# END OF NOTEBOOK

In [814]:
# Find common elements in 2 data collections (here lists)
# 
aoc_ls = [1,2,3,4,5,6,7,8,9,10]
mgt_ls = [8,9,10,11,12,13,14,15,16,17,18]

common_ls = list(set(aoc_ls).intersection(set(mgt_ls)))

[8, 9, 10]

In [ ]:
for auser in common_ls:
    user (user_fields=['created_at','public_metrics'])

In [ ]:
# Twitter Standard API v1.1
# Get Tweet Timelines
# https://developer.twitter.com/en/docs/twitter-api/v1/tweets/timelines/api-reference/get-statuses-user_timeline

api.user_timeline('GovEvers')

In [38]:
govs_tweets_df = pd.DataFrame()

for i, agov_username in enumerate(govs_username_ls[:3]):
    print(f'Processing #{i}: {agov_username}')
    get_user_tweets(x.Twitter_username, x.Political_party),axis=1)
gc.collect()
    # govs_tweets_df.append(agov_username)
    # govs_df.apply(lambda x:get_user_tweets(x.Twitter_username, x.Political_party),axis=1)
    df.apply(lambda x:get_user_tweets(x.Twitter_username, x.Political_party),axis=1)
gc.collect()
    gc.collect()

Processing #0: GovEvers


26

Processing #1: TinaKotek


0

Processing #2: GovSisolak


0